# Predicting User Churn Using Machine-Learning

#### Problem

##### User churn is a leading problem for our clients, with it being 5-25 times more costly to acquire a customer compared to retaining an existing customer. The following exercise  produces a model that predicts whether a user is going to churn based on their streaming and misc. data over a 3 month period.

#### Data

##### Data comes from a dataset of plays and views of approximatly 3000 users between 17th June and 17th September, with top-level data cleansing performed within SQL (such as allocation of plays to each rail and inclusion of metadata). All other data cleansing is performed below. The data is aggregated into month sections. A description of the data available can be found below.

- user_primaryid - this is a unique identifier given to each user
- action_time_frame - the time frame in question - given as days away from the end of the period
- ActionandAdventure - the total plays going to Action content
- Comedy - the total plays going to Comedy content
- Documentary - the total plays going to Documentary content
- Drama	- the total plays going to Drama content
- Erotic - the total plays going to Erotic content
- Horror - the total plays going to Horror content
- Indie	- the total plays going to Indie content
- KidsandFamily	- the total plays going to Kids content
- Romance - the total plays going to Romance content
- ScienceFictionandFantasy - the total plays going to Sci-Fi content
- Stand-Up - the total plays going to Stand-Up content
- Thriller - the total plays going to Thriller content
- Vault - the total plays going to Vault content
- session_bounce_rate - the bounce rate for each user (1 indicates all the user's sessions do not result in plays)
- total_sessions - the total number of unique sessions for each user 
- total_devices - the total number of unique devices used by each user 
- days_accessed	- the total number of days each user used the platform
- avg_percent_stream - the average percent of content streamed
- total_mins_stream	- the total minutes of content streamed
- total_brands_watched - the total number of unique brands watched
- age_newest_content - the age of the most recent piece of content watched
- recs_used - the total number of recommendations used
- percent_mins_movies - the proportion of content watched that are movies
- approx_series_completion - the approximate series completion for each user
- sd_percent - the proportion of content watched that is in SD

#### Sections

1. EDA - investigate metadata and beacons dataset to understand what features are possible.
2. Implement features (see 'Features Desired')
3. Reshaping - Reshape the dataset so its more suitable for machine learning - the idea is to compress the data into columns e.g. plays within x days, views within x days to make later machine learning more easy
4. Date Cleansing - perform data cleansing techniques learnt during the course on the raw data
5. Machine Learning

#### Beacons SQL Code

WITH epix_beacons AS (
SELECT 
action,
datepartition,
thing_id,
custom_rule_id,
identifier_eventid,
b.user_primaryid,
CAST(SUBSTRING(REPLACE(b.timestamp_initiated,'T',' '),1,19) AS timestamp) as timestamp_action,
MAX(b.custom_epix_streamdurationpercent) OVER (PARTITION BY b.user_primaryid, b.thing_id ORDER BY b.timestamp_initiated ASC ROWS BETWEEN CURRENT ROW AND 3 FOLLOWING) AS pp_percent_streamed,
MAX(b.custom_epix_streamdurationseconds) OVER (PARTITION BY b.user_primaryid, b.thing_id ORDER BY b.timestamp_initiated ASC ROWS BETWEEN CURRENT ROW AND 3 FOLLOWING) AS pp_seconds_streamed,

CASE WHEN custom_epix_platform IS NULL THEN
LAG(b.custom_epix_platform) IGNORE NULLS OVER (PARTITION BY b.user_primaryid ORDER BY timestamp_initiated asc) 
ELSE custom_epix_platform END AS custom_epix_platform,

CASE WHEN custom_epix_selection_rail IS NULL THEN
LAG(b.custom_epix_selection_rail) IGNORE NULLS OVER (PARTITION BY b.user_primaryid ORDER BY timestamp_initiated asc) 
ELSE custom_epix_selection_rail END AS selection_rail
FROM userids_for_churn_v2 u
LEFT JOIN awsdatacatalog.productionepixdatabase.beacon b ON b.user_primaryid = u.user_primaryid
where datepartition >= '2022-06-17' AND datepartition <= '2022-09-17'),

epix_session AS 
(SELECT CONCAT(user_primaryid, session_id_add) AS session_id, identifier_eventid
FROM
(SELECT
CASE WHEN identifier_eventid_start IS NULL AND LAG(identifier_eventid_start) IGNORE NULLS OVER (PARTITION BY user_primaryid, custom_epix_platform ORDER BY timestamp_action ASC) IS NOT NULL THEN LAG(identifier_eventid_start) IGNORE NULLS OVER (PARTITION BY user_primaryid, custom_epix_platform ORDER BY timestamp_action ASC) WHEN identifier_eventid_start IS NOT NULL THEN identifier_eventid_start ELSE '0' END AS session_id_add, user_primaryid, timestamp_action, identifier_eventid, custom_epix_platform
FROM
(SELECT CASE WHEN DATE_DIFF('hour', LAG(CAST(timestamp_action AS timestamp)) OVER (PARTITION BY user_primaryid ORDER BY timestamp_action ASC), CAST(timestamp_action AS timestamp)) > 3 THEN identifier_eventid END AS identifier_eventid_start, user_primaryid, timestamp_action, identifier_eventid, custom_epix_platform 
FROM epix_beacons))),

epix_metadata AS  
(SELECT 
  CAST(CAST(coalesce(regexp_extract(regexp_extract(m.thing_duration, '\d+H'), '\d+'),'0') AS integer)*60 + cast(coalesce(regexp_extract(regexp_extract(m.thing_duration, '\d+M'), '\d+'),'0') AS integer) AS int) AS thing_duration, 
  thing_id
  FROM awsdatacatalog.productionepixdatabase.metadata m),
  
epix_plays AS (
SELECT
a.*,
m.thing_duration,
s.session_id,
CASE WHEN action = 'view' THEN NULL WHEN pp_percent_streamed > 100 THEN 1 WHEN pp_percent_streamed IS NULL THEN 0 ELSE ROUND(CAST(pp_percent_streamed AS double) / 100, 4) END AS percent_stream,
CASE WHEN action = 'view' THEN NULL WHEN pp_percent_streamed > 100 THEN CAST(thing_duration AS double) WHEN pp_percent_streamed IS NULL THEN 0 ELSE ROUND(CAST(pp_seconds_streamed AS double) / 60, 1) END AS mins_stream
FROM epix_beacons a
LEFT JOIN epix_metadata m ON a.thing_id = m.thing_id
LEFT JOIN epix_session s ON a.identifier_eventid = s.identifier_eventid
WHERE action IN ('play', 'view'))

SELECT *
FROM epix_plays

#### UserIDs SQL Code

WITH churn_ids AS (SELECT user_primaryid, 1 AS user_has_churned
FROM
(SELECT DISTINCT user_primaryid
FROM beacon
WHERE datepartition >= '2022-09-17' AND custom_epix_accountstatus = 'Churn' AND user_primaryid IS NOT NULL)),

non_churn_ids AS (SELECT user_primaryid, 0 AS user_has_churned
FROM
(SELECT DISTINCT b.user_primaryid
FROM beacon b
LEFT JOIN churn_ids c ON c.user_primaryid = b.user_primaryid
WHERE c.user_primaryid IS NULL AND action = 'play' AND custom_epix_accountstatus = 'Subscriber'
LIMIT 22200)),

all_users AS (SELECT DISTINCT user_primaryid, user_has_churned
FROM churn_ids
UNION ALL
SELECT DISTINCT user_primaryid, user_has_churned
FROM non_churn_ids),

user_billing_cycles AS (SELECT user_primaryid, CASE WHEN billing_cycles IS NULL OR billing_cycles = 0 THEN 1 ELSE billing_cycles END AS billing_cycles
FROM
(SELECT DISTINCT u.user_primaryid, COUNT(DISTINCT b.custom_epix_billingcycle) AS billing_cycles
FROM all_users u
LEFT JOIN beacon b ON u.user_primaryid = b.user_primaryid
WHERE b.datepartition >= '2022-06-17' AND  b.datepartition <= '2022-09-17'
GROUP BY u.user_primaryid))
,

user_first_sign_in AS (
SELECT DISTINCT u.user_primaryid, MIN(b.datepartition) OVER (PARTITION BY b.user_primaryid) AS first_sign_in
FROM all_users u 
LEFT JOIN beacon b ON u.user_primaryid = b.user_primaryid
),

user_stats AS (
SELECT user_primaryid, user_has_churned, custom_epix_billingcycle, custom_epix_billingprovider, custom_epix_productprice, CASE WHEN custom_epix_freetrialtype IS NOT NULL THEN 1 ELSE 0 END AS user_had_free_trial
FROM 
(SELECT user_primaryid, user_has_churned, 
    CASE WHEN custom_epix_billingcycle IS NULL AND lag_custom_epix_billingcycle IS NOT NULL THEN lag_custom_epix_billingcycle WHEN lead_custom_epix_billingcycle IS NOT NULL THEN lead_custom_epix_billingcycle ELSE custom_epix_billingcycle END AS custom_epix_billingcycle, 
    CASE WHEN custom_epix_productprice IS NULL AND lag_custom_epix_productprice IS NOT NULL THEN lag_custom_epix_productprice WHEN lead_custom_epix_productprice IS NOT NULL THEN lead_custom_epix_productprice ELSE custom_epix_productprice END AS custom_epix_productprice, 
    CASE WHEN custom_epix_freetrialtype IS NULL AND lag_custom_epix_freetrialtype IS NOT NULL THEN lag_custom_epix_freetrialtype WHEN lead_custom_epix_freetrialtype IS NOT NULL THEN lead_custom_epix_freetrialtype ELSE custom_epix_freetrialtype END AS custom_epix_freetrialtype, 
    CASE WHEN custom_epix_billingprovider IS NULL AND lag_custom_epix_billingprovider IS NOT NULL THEN lag_custom_epix_billingprovider WHEN lead_custom_epix_billingprovider IS NOT NULL THEN lead_custom_epix_billingprovider ELSE custom_epix_billingprovider END AS custom_epix_billingprovider, 
    ROW_NUMBER() OVER (PARTITION BY user_primaryid) AS ROW_NUMBER
FROM
(SELECT user_primaryid, user_has_churned, LAG(custom_epix_billingcycle) IGNORE NULLS OVER (PARTITION BY user_primaryid) AS lag_custom_epix_billingcycle, LEAD(custom_epix_billingcycle) IGNORE NULLS OVER (PARTITION BY user_primaryid) AS lead_custom_epix_billingcycle, LAG(custom_epix_productprice) IGNORE NULLS OVER (PARTITION BY user_primaryid) AS lag_custom_epix_productprice, LEAD(custom_epix_productprice) IGNORE NULLS OVER (PARTITION BY user_primaryid) AS lead_custom_epix_productprice, LAG(custom_epix_freetrialtype) IGNORE NULLS OVER (PARTITION BY user_primaryid) AS lag_custom_epix_freetrialtype, LEAD(custom_epix_freetrialtype) IGNORE NULLS OVER (PARTITION BY user_primaryid) AS lead_custom_epix_freetrialtype, custom_epix_billingcycle, custom_epix_productprice, custom_epix_freetrialtype, custom_epix_billingprovider, LAG(custom_epix_billingprovider) IGNORE NULLS OVER (PARTITION BY user_primaryid) AS lag_custom_epix_billingprovider, LEAD(custom_epix_billingprovider) IGNORE NULLS OVER (PARTITION BY user_primaryid) AS lead_custom_epix_billingprovider

FROM
              (SELECT u.user_primaryid, 
              u.user_has_churned, 
              b.custom_epix_billingcycle, 
              b.custom_epix_productprice,
              b.custom_epix_billingprovider,
              b.custom_epix_freetrialtype
FROM all_users u 
LEFT JOIN beacon b ON u.user_primaryid = b.user_primaryid
GROUP BY u.user_primaryid, u.user_has_churned, b.custom_epix_billingprovider, b.custom_epix_billingcycle, b.custom_epix_productprice, b.custom_epix_freetrialtype)))
WHERE ROW_NUMBER = 1)

SELECT u.*, f.first_sign_in
FROM user_stats u 
LEFT JOIN user_first_sign_in f ON f.user_primaryid = u.user_primaryid

In [1]:
# Import Packages
import pandas as pd
import datetime as dt
import numpy as np

### Section 1 - EDA

In [2]:
# Load Data
metadata = pd.read_csv('/Users/mattcadel/Documents/Python/DSML/metadata.csv')
metadata.head(10)


,thing_duration,thing_id,thing_contentrating_custom_rawratingvalue,thing_video_videoquality,thing_typename,thing_genre_name,thing_seasonnumber,thing_name,thing_contentrating_name,thing_custom_categories_name,...,thing_producer_custom_type,thing_director_name,thing_brandid,thing_partofseries_name,thing_keywords_name,thing_partofseason_name,thing_inlanguage,thing_publication_startdate,thing_numberofseasons,thing_episodenumber
0,122,bW92aWU7MTY5Mg==,[PG-13],HD,Movie,[Drama],NaN,Moll Flanders,[MPAA],NaN,...,"[crew, crew]",[Pen Densham],NaN,NaN,NaN,NaN,en,[2020-04-01T04:00:00+00:00],NaN,NaN
1,0,c2VyaWVzOzEwMDM=,[TV-MA],NaN,TVSeries,[Documentary],NaN,Road To The NHL,[TV],NaN,...,"[crew, crew, crew, crew, crew, crew, crew, cre...",NaN,NaN,NaN,NaN,NaN,en,NaN,6.0,NaN
2,89,bW92aWU7MTEwNw==,[TV-14],HD,Movie,[Vault],NaN,Carry On Columbus,[TV],NaN,...,"[crew, crew]",[Gerald Thomas],bW92aWU7MTEwNw==,NaN,NaN,NaN,en,[2019-03-01T05:00:01+00:00],NaN,NaN
3,86,bW92aWU7NTAyMQ==,[PG-13],HD,Movie,[Comedy],NaN,Fun Size,[MPAA],NaN,...,"[crew, crew, crew, crew]",[Josh Schwartz],NaN,NaN,NaN,NaN,en,[2019-06-01T04:00:01+00:00],NaN,NaN
4,80,bW92aWU7NDU4Nw==,[TV-PG],HD,Movie,[Vault],NaN,Riders to the Stars,[TV],[Vault],...,"[crew, crew]",[Richard Carlson],bW92aWU7NDU4Nw==,NaN,NaN,NaN,en,[2019-03-01T05:00:01+00:00],NaN,NaN
5,81,bW92aWU7MjQwMg==,[TV-PG],SD,Movie,[Vault],NaN,Curse of the Swamp Creature,[TV],NaN,...,"[crew, crew]",[Larry Buchanan],bW92aWU7MjQwMg==,NaN,NaN,NaN,en,[2019-03-01T05:00:01+00:00],NaN,NaN
6,87,bW92aWU7MTI2OTg=,[NR],HD,Movie,[Vault],NaN,Primitive London,[MPAA],NaN,...,"[crew, crew]",[Arnold L. Miller],NaN,NaN,NaN,NaN,en,[2019-09-01T04:00:01+00:00],NaN,NaN
7,72,bW92aWU7MjY5NA==,[TV-PG],HD,Movie,[Vault],NaN,Riot in Juvenile Prison,[TV],[Vault],...,[crew],[Edward L. Cahn],bW92aWU7MjY5NA==,NaN,NaN,NaN,en,[2019-03-01T05:00:01+00:00],NaN,NaN
8,94,bW92aWU7MzU4MQ==,[TV-PG],HD,Movie,[Vault],NaN,Timestalkers,[TV],[Vault],...,"[crew, crew, crew, crew]",[Michael Schultz],bW92aWU7MzU4MQ==,NaN,NaN,NaN,en,[2019-03-01T05:00:01+00:00],NaN,NaN
9,84,bW92aWU7MTI3MTc=,[R],HD,Movie,[Vault],NaN,The Search for One Eyed Jimmy,[MPAA],NaN,...,NaN,[Sam Henry Kass],NaN,NaN,NaN,NaN,en,[2019-09-01T04:00:01+00:00],NaN,NaN


In [3]:
# Available Columns
metadata.columns

Index(['thing_duration', 'thing_id',
       'thing_contentrating_custom_rawratingvalue', 'thing_video_videoquality',
       'thing_typename', 'thing_genre_name', 'thing_seasonnumber',
       'thing_name', 'thing_contentrating_name',
       'thing_custom_categories_name', 'thing_datepublished',
       'thing_actor_name', 'thing_producer_name', 'thing_producer_custom_type',
       'thing_director_name', 'thing_brandid', 'thing_partofseries_name',
       'thing_keywords_name', 'thing_partofseason_name', 'thing_inlanguage',
       'thing_publication_startdate', 'thing_numberofseasons',
       'thing_episodenumber'],
      dtype='object')

In [4]:
metadata.describe()

,thing_duration,thing_seasonnumber,thing_numberofseasons,thing_episodenumber
count,4143.000000,465.000000,38.000000,465.000000
mean,83.529327,1.632258,1.815789,8.531183
std,25.768798,0.893120,1.111494,36.881795
min,0.000000,1.000000,1.000000,1.000000
25%,66.000000,1.000000,1.000000,3.000000
50%,88.000000,1.000000,1.000000,5.000000
75%,99.000000,2.000000,2.000000,7.000000
max,245.000000,5.000000,6.000000,404.000000


##### Null Values 
- thing_seasonnumber is often null - hence will require cleaning
- thing_genre_name should be used over thing_custom_categories_name
- thing_brandid is sometimes null - hence will use thing_id when null
- Actor / Producer will not be used as believe will introduce too much variation and won't necessarily help



In [5]:
metadata.isnull().sum()

thing_duration                                  0
thing_id                                        0
thing_contentrating_custom_rawratingvalue       1
thing_video_videoquality                      169
thing_typename                                  0
thing_genre_name                               16
thing_seasonnumber                           3678
thing_name                                      0
thing_contentrating_name                        0
thing_custom_categories_name                  966
thing_datepublished                             0
thing_actor_name                              516
thing_producer_name                           430
thing_producer_custom_type                    430
thing_director_name                           192
thing_brandid                                 938
thing_partofseries_name                      3678
thing_keywords_name                          3853
thing_partofseason_name                      3694
thing_inlanguage                                0


In [6]:
metadata['thing_custom_categories_name'].value_counts()


[Vault]                                                                 1865
[Drama]                                                                  231
[Documentary]                                                            184
[Comedy]                                                                 168
[Action and Adventure]                                                    85
                                                                        ... 
[Drama, Science Fiction and Fantasy, Romance]                              1
[Science Fiction and Fantasy, Kids and Family, Action and Adventure]       1
[Drama, Science Fiction and Fantasy, Kids and Family]                      1
[Horror, Science Fiction and Fantasy, Thriller]                            1
[Vault, Thriller]                                                          1
Name: thing_custom_categories_name, Length: 133, dtype: int64

In [7]:
metadata.groupby(['thing_numberofseasons', 'thing_typename'])['thing_numberofseasons'].count()


thing_numberofseasons  thing_typename
1.0                    TVSeries          20
2.0                    TVSeries           9
3.0                    TVSeries           7
4.0                    TVSeries           1
6.0                    TVSeries           1
Name: thing_numberofseasons, dtype: int64

In [8]:
metadata.thing_actor_name.isnull().groupby(metadata['thing_typename']).sum().astype(int).reset_index(name='count')


,thing_typename,count
0,Episode,452
1,Movie,45
2,TVSeries,19


In [9]:
metadata.thing_director_name.isnull().groupby(metadata['thing_typename']).sum().astype(int).reset_index(name='count')


,thing_typename,count
0,Episode,155
1,Movie,14
2,TVSeries,23


In [10]:
metadata.groupby(['thing_numberofseasons', 'thing_typename'])['thing_numberofseasons'].count()


thing_numberofseasons  thing_typename
1.0                    TVSeries          20
2.0                    TVSeries           9
3.0                    TVSeries           7
4.0                    TVSeries           1
6.0                    TVSeries           1
Name: thing_numberofseasons, dtype: int64

In [11]:
metadata.groupby('thing_inlanguage')['thing_inlanguage'].count()

thing_inlanguage
en    4143
Name: thing_inlanguage, dtype: int64

In [12]:
metadata.thing_actor_name.isnull().groupby(metadata['thing_typename']).sum().astype(int).reset_index(name='count')


,thing_typename,count
0,Episode,452
1,Movie,45
2,TVSeries,19


#### EDA Findings

- Multiple Genres per thing
- Genre more populated than category (so will be used)
- Only 'TVSeries' thing_typename have thing_numberofseasons populated (hence will require data cleansing to add to individual episodes)
- Whole of catalouge is in English
- thing_director_name is often null but will use TVSeries to add account for as many nulls as possible

## Feature Engineering

#### Adding No. Series to each episode

In [13]:
number_series = metadata.loc[metadata['thing_typename'] == 'TVSeries'].groupby('thing_brandid')['thing_numberofseasons'].max().to_frame().reset_index().rename(columns={'thing_numberofseasons':'number_of_seasons'})
metadata = pd.merge(metadata,number_series[['number_of_seasons','thing_brandid']],on='thing_brandid', how='left')
metadata

,thing_duration,thing_id,thing_contentrating_custom_rawratingvalue,thing_video_videoquality,thing_typename,thing_genre_name,thing_seasonnumber,thing_name,thing_contentrating_name,thing_custom_categories_name,...,thing_director_name,thing_brandid,thing_partofseries_name,thing_keywords_name,thing_partofseason_name,thing_inlanguage,thing_publication_startdate,thing_numberofseasons,thing_episodenumber,number_of_seasons
0,122,bW92aWU7MTY5Mg==,[PG-13],HD,Movie,[Drama],NaN,Moll Flanders,[MPAA],NaN,...,[Pen Densham],NaN,NaN,NaN,NaN,en,[2020-04-01T04:00:00+00:00],NaN,NaN,NaN
1,0,c2VyaWVzOzEwMDM=,[TV-MA],NaN,TVSeries,[Documentary],NaN,Road To The NHL,[TV],NaN,...,NaN,NaN,NaN,NaN,NaN,en,NaN,6.0,NaN,NaN
2,89,bW92aWU7MTEwNw==,[TV-14],HD,Movie,[Vault],NaN,Carry On Columbus,[TV],NaN,...,[Gerald Thomas],bW92aWU7MTEwNw==,NaN,NaN,NaN,en,[2019-03-01T05:00:01+00:00],NaN,NaN,NaN
3,86,bW92aWU7NTAyMQ==,[PG-13],HD,Movie,[Comedy],NaN,Fun Size,[MPAA],NaN,...,[Josh Schwartz],NaN,NaN,NaN,NaN,en,[2019-06-01T04:00:01+00:00],NaN,NaN,NaN
4,80,bW92aWU7NDU4Nw==,[TV-PG],HD,Movie,[Vault],NaN,Riders to the Stars,[TV],[Vault],...,[Richard Carlson],bW92aWU7NDU4Nw==,NaN,NaN,NaN,en,[2019-03-01T05:00:01+00:00],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4138,109,bW92aWU7MTk1NTY=,[R],HD,Movie,[Action and Adventure],NaN,The Professional,[MPAA],[Action and Adventure],...,[Luc Besson],bW92aWU7MTk1NTY=,NaN,"[written and directed by, box office success, ...",NaN,en,[2022-11-01T04:00:01+00:00],NaN,NaN,NaN
4139,69,bW92aWU7MTk1Nzg=,[TV-MA],HD,Movie,[Vault],NaN,Rook,[TV],[Vault],...,[Stephen Morgan],bW92aWU7MTk1Nzg=,NaN,NaN,NaN,en,[2022-04-01T04:00:01+00:00],NaN,NaN,NaN
4140,27,ZXBpc29kZTsxMzQ1,[TV-MA],HD,Episode,[Drama],1.0,Bridge and Tunnel (S1 E1): The Graduates,[TV],[Drama],...,[Edward Burns],c2VyaWVzOzEwMjg=,Bridge and Tunnel,NaN,Bridge and Tunnel,en,[2021-01-24T05:00:01+00:00],NaN,1.0,2.0
4141,58,ZXBpc29kZTsxMzQ2,[TV-PG],HD,Episode,[Documentary],1.0,The Eisen Hour (S1 E1): Episode One,[TV],[Documentary],...,NaN,c2VyaWVzOzEwMjk=,The Eisen Hour,NaN,The Eisen Hour,en,[2021-01-24T05:00:00+00:00],NaN,1.0,1.0


In [14]:
metadata.thing_contentrating_custom_rawratingvalue

0       [PG-13]
1       [TV-MA]
2       [TV-14]
3       [PG-13]
4       [TV-PG]
         ...   
4138        [R]
4139    [TV-MA]
4140    [TV-MA]
4141    [TV-PG]
4142    [TV-MA]
Name: thing_contentrating_custom_rawratingvalue, Length: 4143, dtype: object

#### Converting thing_publication_startdate into datetime format

In [15]:
metadata['thing_publication_startdate'] = pd.to_datetime(metadata['thing_publication_startdate'].str[1:11])
metadata['thing_publication_startdate'].dt.month

0        4.0
1        NaN
2        3.0
3        6.0
4        3.0
        ... 
4138    11.0
4139     4.0
4140     1.0
4141     1.0
4142    10.0
Name: thing_publication_startdate, Length: 4143, dtype: float64

In [16]:
beacons = pd.read_csv('/Users/mattcadel/Documents/Python/DSML/beacons.csv')
beacons.head(20)
# beacons.columnsz

,action,datepartition,thing_id,custom_rule_id,identifier_eventid,user_primaryid,timestamp_action,pp_percent_streamed,pp_seconds_streamed,custom_epix_platform,selection_rail,thing_duration,session_id,percent_stream,mins_stream
0,view,2022-07-09,NaN,NaN,9b67152f-902f-4439-92ef-26618ae69f26,856ce74f9d431ac080bd466b08315883da7ffd7eb2e692...,2022-07-09 03:46:34.000,NaN,NaN,Fire TV,Movies,NaN,856ce74f9d431ac080bd466b08315883da7ffd7eb2e692...,NaN,NaN
1,play,2022-07-08,ZXBpc29kZTsxMjA2,NaN,59ed1fc7-7c4d-4cf7-aabf-d632b35e9c79,a8abd348e399e19678df3a41be84eb523805c4335d92c3...,2022-07-08 00:26:01.000,NaN,NaN,Roku,Home,45.0,a8abd348e399e19678df3a41be84eb523805c4335d92c3...,0.0000,0.0
2,play,2022-08-19,ZXBpc29kZTsxMjA2,NaN,c630fb69-f8c1-41cb-99a3-8fc28032f316,a8abd348e399e19678df3a41be84eb523805c4335d92c3...,2022-08-19 23:09:09.000,NaN,NaN,Roku,Home,45.0,a8abd348e399e19678df3a41be84eb523805c4335d92c3...,0.0000,0.0
3,view,2022-06-17,NaN,NaN,2cf01604-0748-4f90-bdc2-0234755d0549,fa3a43d2e1f0baee7d10905e0bd0b6bf3a8f79c4aed059...,2022-06-17 00:20:44.000,NaN,NaN,Roku,Movies,NaN,fa3a43d2e1f0baee7d10905e0bd0b6bf3a8f79c4aed059...,NaN,NaN
4,play,2022-07-04,ZXBpc29kZTsxODAy,NaN,56d42942-ae86-466b-ab97-ed0a8c0cd1da,fc68cfc3cb47af751db6a0019bbf528dd718ae3320a4c4...,2022-07-04 02:49:53.000,3.066,84.0,Apple TV,Home,45.0,fc68cfc3cb47af751db6a0019bbf528dd718ae3320a4c4...,0.0307,1.4
5,view,2022-08-11,bW92aWU7MjMwOA==,NaN,79379971-b1bb-4152-9f9c-6101e19d0fa1,fc68cfc3cb47af751db6a0019bbf528dd718ae3320a4c4...,2022-08-11 23:46:49.000,NaN,NaN,Apple Mobile,Home,92.0,fc68cfc3cb47af751db6a0019bbf528dd718ae3320a4c4...,NaN,NaN
6,view,2022-07-29,bW92aWU7NTA5MQ==,NaN,a93b09d7-fbe8-41c9-a792-6623d16578c1,fc68cfc3cb47af751db6a0019bbf528dd718ae3320a4c4...,2022-07-29 04:31:06.000,NaN,NaN,Apple Mobile,Home,87.0,fc68cfc3cb47af751db6a0019bbf528dd718ae3320a4c4...,NaN,NaN
7,play,2022-09-04,ZXBpc29kZTsxODMw,NaN,8a595a4f-7407-4cd3-94ab-5cebabb186b7,a27aa0bc54015d54bac387bdd4a00b3695266720671d64...,2022-09-04 01:33:26.000,99.717,2020.0,Apple TV,Home,33.0,a27aa0bc54015d54bac387bdd4a00b3695266720671d64...,0.9972,33.7
8,view,2022-08-30,bW92aWU7MTM2NjA=,NaN,506f3e5c-aa92-426c-93e5-2ce96ffe26b1,a27aa0bc54015d54bac387bdd4a00b3695266720671d64...,2022-08-30 17:43:36.000,NaN,NaN,Apple TV,Live TV,113.0,a27aa0bc54015d54bac387bdd4a00b3695266720671d64...,NaN,NaN
9,view,2022-09-02,bW92aWU7MTY4MzE=,NaN,f4f78fc7-5732-4638-a4bd-12f3ad57d09d,a27aa0bc54015d54bac387bdd4a00b3695266720671d64...,2022-09-02 18:01:23.000,NaN,NaN,Apple TV,Movies,83.0,a27aa0bc54015d54bac387bdd4a00b3695266720671d64...,NaN,NaN


#### Nulls on custom_epix_platform and selection_rail
- These are not a problem as only the most use platform and rail are needed for the ML

In [17]:
beacons.loc[beacons['action'] == 'play'].isnull().sum()

action                      0
datepartition               0
thing_id                    0
custom_rule_id          20871
identifier_eventid          0
user_primaryid              0
timestamp_action            0
pp_percent_streamed     10938
pp_seconds_streamed     10915
custom_epix_platform       90
selection_rail            446
thing_duration              0
session_id                  0
percent_stream              0
mins_stream                 0
dtype: int64

- Creating view and play column to help with later counting of plays/views for each user
- Getting the hour of each time stamp to identify each user's primary time of dat user

In [18]:
beacons.loc[(beacons['action'] == 'play') , 'play'] = 1
beacons.loc[(beacons['action'] == 'view') , 'view'] = 1
beacons['timestamp_action'] = pd.to_datetime(beacons['timestamp_action'])
beacons['hour'] = beacons['timestamp_action'].dt.hour

beacons.loc[(beacons['hour'] >= 22) & (beacons['hour'] <= 24), 'time_band'] = 'Night'
beacons.loc[(beacons['hour'] >= 0) & (beacons['hour'] <= 3), 'time_band'] = 'Night'
beacons.loc[(beacons['hour'] > 3) & (beacons['hour'] < 7), 'time_band'] = 'Early_Morning'
beacons.loc[(beacons['hour'] >= 17) & (beacons['hour'] < 22), 'time_band'] = 'Evening'
beacons.loc[(beacons['hour'] >= 12) & (beacons['hour'] < 17), 'time_band'] = 'Afternoon'
beacons.loc[(beacons['hour'] >= 7) & (beacons['hour'] < 12), 'time_band'] = 'Morning'

In [19]:
beacons

,action,datepartition,thing_id,custom_rule_id,identifier_eventid,user_primaryid,timestamp_action,pp_percent_streamed,pp_seconds_streamed,custom_epix_platform,selection_rail,thing_duration,session_id,percent_stream,mins_stream,play,view,hour,time_band
0,view,2022-07-09,NaN,NaN,9b67152f-902f-4439-92ef-26618ae69f26,856ce74f9d431ac080bd466b08315883da7ffd7eb2e692...,2022-07-09 03:46:34,NaN,NaN,Fire TV,Movies,NaN,856ce74f9d431ac080bd466b08315883da7ffd7eb2e692...,NaN,NaN,NaN,1.0,3,Night
1,play,2022-07-08,ZXBpc29kZTsxMjA2,NaN,59ed1fc7-7c4d-4cf7-aabf-d632b35e9c79,a8abd348e399e19678df3a41be84eb523805c4335d92c3...,2022-07-08 00:26:01,NaN,NaN,Roku,Home,45.0,a8abd348e399e19678df3a41be84eb523805c4335d92c3...,0.0000,0.0,1.0,NaN,0,Night
2,play,2022-08-19,ZXBpc29kZTsxMjA2,NaN,c630fb69-f8c1-41cb-99a3-8fc28032f316,a8abd348e399e19678df3a41be84eb523805c4335d92c3...,2022-08-19 23:09:09,NaN,NaN,Roku,Home,45.0,a8abd348e399e19678df3a41be84eb523805c4335d92c3...,0.0000,0.0,1.0,NaN,23,Night
3,view,2022-06-17,NaN,NaN,2cf01604-0748-4f90-bdc2-0234755d0549,fa3a43d2e1f0baee7d10905e0bd0b6bf3a8f79c4aed059...,2022-06-17 00:20:44,NaN,NaN,Roku,Movies,NaN,fa3a43d2e1f0baee7d10905e0bd0b6bf3a8f79c4aed059...,NaN,NaN,NaN,1.0,0,Night
4,play,2022-07-04,ZXBpc29kZTsxODAy,NaN,56d42942-ae86-466b-ab97-ed0a8c0cd1da,fc68cfc3cb47af751db6a0019bbf528dd718ae3320a4c4...,2022-07-04 02:49:53,3.066,84.0,Apple TV,Home,45.0,fc68cfc3cb47af751db6a0019bbf528dd718ae3320a4c4...,0.0307,1.4,1.0,NaN,2,Night
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48272,view,2022-07-25,bW92aWU7MTk2MzM=,NaN,bdf47eaf-6d2a-46ff-88fc-6ef0345298e3,56f3e79121bb4ab8d160ac1a1aff5be42b3717d712ce61...,2022-07-25 05:26:17,NaN,NaN,Roku,Home,97.0,56f3e79121bb4ab8d160ac1a1aff5be42b3717d712ce61...,NaN,NaN,NaN,1.0,5,Early_Morning
48273,view,2022-08-27,bW92aWU7MTk2Mzg=,NaN,790393dd-c315-4fa4-824e-74d20a818f70,56f3e79121bb4ab8d160ac1a1aff5be42b3717d712ce61...,2022-08-27 04:51:03,NaN,NaN,Roku,Home,101.0,56f3e79121bb4ab8d160ac1a1aff5be42b3717d712ce61...,NaN,NaN,NaN,1.0,4,Early_Morning
48274,view,2022-07-31,bW92aWU7MTQyNzI=,a0182e54-72ee-4180-8a02-8b4289a64a04,a1596d5a-f03d-4352-8cf8-64cb1d69dd79,c429d063621d86a8e77b8839137870ad57b7b155ef464c...,2022-07-31 22:42:33,13.624,777.0,Fire TV,Movie DP,95.0,c429d063621d86a8e77b8839137870ad57b7b155ef464c...,NaN,NaN,NaN,1.0,22,Night
48275,view,2022-07-23,NaN,NaN,c5fd51b8-8111-4052-ae79-07bea78dc8c8,c429d063621d86a8e77b8839137870ad57b7b155ef464c...,2022-07-23 17:04:34,NaN,NaN,Fire TV,Movies,NaN,c429d063621d86a8e77b8839137870ad57b7b155ef464c...,NaN,NaN,NaN,1.0,17,Evening


- Calculating the days since their previous play at the end of the period
- Assigning each timestamp a grouping e.g. 'Under 30 Days' (based on days away from the end of the period)

In [20]:
# Time Segmenting

beacons['date_from_end'] = round((pd.to_datetime('2022-09-17') - beacons['timestamp_action']) / np.timedelta64(1, 'D'),2)

beacons.loc[(beacons['date_from_end'] <= 30), 'action_time_frame'] = 'Under 30 Days'
beacons.loc[(beacons['date_from_end'] > 30) & (beacons['hour'] <= 60), 'action_time_frame'] = '30 - 60 Days'
beacons.loc[(beacons['date_from_end'] > 60), 'action_time_frame'] = '60+ Days'

set(beacons.action_time_frame)

{'30 - 60 Days', '60+ Days', 'Under 30 Days'}

- Finding each users most used rail, time frame, algorithm and device
- Creating a dataframe of churn statuses for each user

In [21]:
beacons = pd.merge(beacons, metadata, on='thing_id', how='left')

primary_time_band = beacons.groupby(['user_primaryid', 'time_band'])['time_band'].count().sort_values().groupby(level=0).tail(1).to_frame().iloc[: , :0].reset_index().rename(columns={'time_band':'favourite_time'})

primary_algorithm = beacons.groupby(['user_primaryid', 'custom_rule_id'])['custom_rule_id'].count().sort_values().groupby(level=0).tail(1).to_frame().iloc[: , :0].reset_index().rename(columns={'custom_rule_id':'favourite_algorithm'})

primary_rail = beacons.groupby(['user_primaryid', 'selection_rail'])['selection_rail'].count().sort_values().groupby(level=0).tail(1).to_frame().iloc[: , :0].reset_index().rename(columns={'selection_rail':'favourite_rail'})

primary_device = beacons.groupby(['user_primaryid', 'custom_epix_platform'])['custom_epix_platform'].count().sort_values().groupby(level=0).tail(1).to_frame().iloc[: , :0].reset_index().rename(columns={'custom_epix_platform':'most_used_device'})


- Finding users favourite brand

In [22]:
beacons

,action,datepartition,thing_id,custom_rule_id,identifier_eventid,user_primaryid,timestamp_action,pp_percent_streamed,pp_seconds_streamed,custom_epix_platform,...,thing_director_name,thing_brandid,thing_partofseries_name,thing_keywords_name,thing_partofseason_name,thing_inlanguage,thing_publication_startdate,thing_numberofseasons,thing_episodenumber,number_of_seasons
0,view,2022-07-09,NaN,NaN,9b67152f-902f-4439-92ef-26618ae69f26,856ce74f9d431ac080bd466b08315883da7ffd7eb2e692...,2022-07-09 03:46:34,NaN,NaN,Fire TV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
1,play,2022-07-08,ZXBpc29kZTsxMjA2,NaN,59ed1fc7-7c4d-4cf7-aabf-d632b35e9c79,a8abd348e399e19678df3a41be84eb523805c4335d92c3...,2022-07-08 00:26:01,NaN,NaN,Roku,...,NaN,c2VyaWVzOzEwMTg=,Sex Life,NaN,Sex Life,en,2019-07-26,NaN,7.0,3.0
2,play,2022-08-19,ZXBpc29kZTsxMjA2,NaN,c630fb69-f8c1-41cb-99a3-8fc28032f316,a8abd348e399e19678df3a41be84eb523805c4335d92c3...,2022-08-19 23:09:09,NaN,NaN,Roku,...,NaN,c2VyaWVzOzEwMTg=,Sex Life,NaN,Sex Life,en,2019-07-26,NaN,7.0,3.0
3,view,2022-06-17,NaN,NaN,2cf01604-0748-4f90-bdc2-0234755d0549,fa3a43d2e1f0baee7d10905e0bd0b6bf3a8f79c4aed059...,2022-06-17 00:20:44,NaN,NaN,Roku,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
4,play,2022-07-04,ZXBpc29kZTsxODAy,NaN,56d42942-ae86-466b-ab97-ed0a8c0cd1da,fc68cfc3cb47af751db6a0019bbf528dd718ae3320a4c4...,2022-07-04 02:49:53,3.066,84.0,Apple TV,...,[David Frazee],c2VyaWVzOzEwNDI=,Billy the Kid,NaN,Billy the Kid,en,2022-06-05,NaN,8.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48272,view,2022-07-25,bW92aWU7MTk2MzM=,NaN,bdf47eaf-6d2a-46ff-88fc-6ef0345298e3,56f3e79121bb4ab8d160ac1a1aff5be42b3717d712ce61...,2022-07-25 05:26:17,NaN,NaN,Roku,...,[Brea Grant],bW92aWU7MTk2MzM=,NaN,NaN,NaN,en,2022-07-22,NaN,NaN,NaN
48273,view,2022-08-27,bW92aWU7MTk2Mzg=,NaN,790393dd-c315-4fa4-824e-74d20a818f70,56f3e79121bb4ab8d160ac1a1aff5be42b3717d712ce61...,2022-08-27 04:51:03,NaN,NaN,Roku,...,"[Reid Carolin, Channing Tatum]",bW92aWU7MTk2Mzg=,NaN,NaN,NaN,en,2022-09-16,NaN,NaN,NaN
48274,view,2022-07-31,bW92aWU7MTQyNzI=,a0182e54-72ee-4180-8a02-8b4289a64a04,a1596d5a-f03d-4352-8cf8-64cb1d69dd79,c429d063621d86a8e77b8839137870ad57b7b155ef464c...,2022-07-31 22:42:33,13.624,777.0,Fire TV,...,[Henry Hobson],bW92aWU7MTQyNzI=,NaN,"[directorial debut, mixed reviews]",NaN,en,2021-11-01,NaN,NaN,NaN
48275,view,2022-07-23,NaN,NaN,c5fd51b8-8111-4052-ae79-07bea78dc8c8,c429d063621d86a8e77b8839137870ad57b7b155ef464c...,2022-07-23 17:04:34,NaN,NaN,Fire TV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN


In [23]:
beacons['thing_brandid'] = beacons['thing_brandid'].fillna(beacons['thing_id'])

primary_brand = beacons.loc[(beacons['action'] == 'play') & (beacons['percent_stream'] >= 0.5)].groupby(['user_primaryid', 'thing_brandid'])['thing_brandid'].count().sort_values().groupby(level=0).tail(1).to_frame().iloc[: , :0].reset_index().rename(columns={'thing_brandid':'favourite_brand'})
beacons = pd.merge(beacons,primary_brand[['user_primaryid','favourite_brand']],on='user_primaryid', how='left')

beacons

primary_brand['favourite_brand'].nunique()


110

- Finding users' most recent and first play, then converting this into weeks from the end of the period

In [24]:
most_recent_play = beacons.loc[(beacons['action'] == 'play')].groupby('user_primaryid')['timestamp_action'].max().to_frame().rename(columns={'timestamp_action':'most_recent_play'}).reset_index()
first_period_play = beacons.loc[(beacons['action'] == 'play')].groupby('user_primaryid')['timestamp_action'].min().to_frame().rename(columns={'timestamp_action':'first_period_play'}).reset_index()
first_period_play

most_recent_play['weeks_from_end'] = round((pd.to_datetime('2022-09-17') - most_recent_play['most_recent_play']) / np.timedelta64(1, 'W'),0)

first_period_play['weeks_from_end'] = round((pd.to_datetime('2022-09-17') - first_period_play['first_period_play']) / np.timedelta64(1, 'W'),0)
first_period_play


beacons['user_primaryid'].nunique()



776

- Creating dummy variables for each genre each play has

In [25]:
beacons['thing_genre_name'] = beacons['thing_genre_name'].str.replace(']', '').str.replace('[', '').str.replace(' ', '').str.split(',')

genres = pd.get_dummies(beacons['thing_genre_name'].apply(pd.Series).stack()).sum(level=0)
beacons = pd.concat([beacons, genres], axis=1)
# genres
# beacons.columns
beacons


/var/folders/72/hqzcd6_j53s1pz09bwht8gmw0000gp/T/ipykernel_92556/606621655.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  beacons['thing_genre_name'] = beacons['thing_genre_name'].str.replace(']', '').str.replace('[', '').str.replace(' ', '').str.split(',')
/var/folders/72/hqzcd6_j53s1pz09bwht8gmw0000gp/T/ipykernel_92556/606621655.py:3: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  genres = pd.get_dummies(beacons['thing_genre_name'].apply(pd.Series).stack()).sum(level=0)


,action,datepartition,thing_id,custom_rule_id,identifier_eventid,user_primaryid,timestamp_action,pp_percent_streamed,pp_seconds_streamed,custom_epix_platform,...,Drama,Erotic,Horror,Indie,KidsandFamily,Romance,ScienceFictionandFantasy,Stand-Up,Thriller,Vault
0,view,2022-07-09,NaN,NaN,9b67152f-902f-4439-92ef-26618ae69f26,856ce74f9d431ac080bd466b08315883da7ffd7eb2e692...,2022-07-09 03:46:34,NaN,NaN,Fire TV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,play,2022-07-08,ZXBpc29kZTsxMjA2,NaN,59ed1fc7-7c4d-4cf7-aabf-d632b35e9c79,a8abd348e399e19678df3a41be84eb523805c4335d92c3...,2022-07-08 00:26:01,NaN,NaN,Roku,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,play,2022-08-19,ZXBpc29kZTsxMjA2,NaN,c630fb69-f8c1-41cb-99a3-8fc28032f316,a8abd348e399e19678df3a41be84eb523805c4335d92c3...,2022-08-19 23:09:09,NaN,NaN,Roku,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,view,2022-06-17,NaN,NaN,2cf01604-0748-4f90-bdc2-0234755d0549,fa3a43d2e1f0baee7d10905e0bd0b6bf3a8f79c4aed059...,2022-06-17 00:20:44,NaN,NaN,Roku,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,play,2022-07-04,ZXBpc29kZTsxODAy,NaN,56d42942-ae86-466b-ab97-ed0a8c0cd1da,fc68cfc3cb47af751db6a0019bbf528dd718ae3320a4c4...,2022-07-04 02:49:53,3.066,84.0,Apple TV,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48272,view,2022-07-25,bW92aWU7MTk2MzM=,NaN,bdf47eaf-6d2a-46ff-88fc-6ef0345298e3,56f3e79121bb4ab8d160ac1a1aff5be42b3717d712ce61...,2022-07-25 05:26:17,NaN,NaN,Roku,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48273,view,2022-08-27,bW92aWU7MTk2Mzg=,NaN,790393dd-c315-4fa4-824e-74d20a818f70,56f3e79121bb4ab8d160ac1a1aff5be42b3717d712ce61...,2022-08-27 04:51:03,NaN,NaN,Roku,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48274,view,2022-07-31,bW92aWU7MTQyNzI=,a0182e54-72ee-4180-8a02-8b4289a64a04,a1596d5a-f03d-4352-8cf8-64cb1d69dd79,c429d063621d86a8e77b8839137870ad57b7b155ef464c...,2022-07-31 22:42:33,13.624,777.0,Fire TV,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
48275,view,2022-07-23,NaN,NaN,c5fd51b8-8111-4052-ae79-07bea78dc8c8,c429d063621d86a8e77b8839137870ad57b7b155ef464c...,2022-07-23 17:04:34,NaN,NaN,Fire TV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
beacons.loc[(beacons['action'] == 'play') & (beacons['thing_video_videoquality'] == 'SD'), 'sd_play'] = 1
beacons


,action,datepartition,thing_id,custom_rule_id,identifier_eventid,user_primaryid,timestamp_action,pp_percent_streamed,pp_seconds_streamed,custom_epix_platform,...,Erotic,Horror,Indie,KidsandFamily,Romance,ScienceFictionandFantasy,Stand-Up,Thriller,Vault,sd_play
0,view,2022-07-09,NaN,NaN,9b67152f-902f-4439-92ef-26618ae69f26,856ce74f9d431ac080bd466b08315883da7ffd7eb2e692...,2022-07-09 03:46:34,NaN,NaN,Fire TV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,play,2022-07-08,ZXBpc29kZTsxMjA2,NaN,59ed1fc7-7c4d-4cf7-aabf-d632b35e9c79,a8abd348e399e19678df3a41be84eb523805c4335d92c3...,2022-07-08 00:26:01,NaN,NaN,Roku,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,play,2022-08-19,ZXBpc29kZTsxMjA2,NaN,c630fb69-f8c1-41cb-99a3-8fc28032f316,a8abd348e399e19678df3a41be84eb523805c4335d92c3...,2022-08-19 23:09:09,NaN,NaN,Roku,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,view,2022-06-17,NaN,NaN,2cf01604-0748-4f90-bdc2-0234755d0549,fa3a43d2e1f0baee7d10905e0bd0b6bf3a8f79c4aed059...,2022-06-17 00:20:44,NaN,NaN,Roku,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,play,2022-07-04,ZXBpc29kZTsxODAy,NaN,56d42942-ae86-466b-ab97-ed0a8c0cd1da,fc68cfc3cb47af751db6a0019bbf528dd718ae3320a4c4...,2022-07-04 02:49:53,3.066,84.0,Apple TV,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48272,view,2022-07-25,bW92aWU7MTk2MzM=,NaN,bdf47eaf-6d2a-46ff-88fc-6ef0345298e3,56f3e79121bb4ab8d160ac1a1aff5be42b3717d712ce61...,2022-07-25 05:26:17,NaN,NaN,Roku,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
48273,view,2022-08-27,bW92aWU7MTk2Mzg=,NaN,790393dd-c315-4fa4-824e-74d20a818f70,56f3e79121bb4ab8d160ac1a1aff5be42b3717d712ce61...,2022-08-27 04:51:03,NaN,NaN,Roku,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
48274,view,2022-07-31,bW92aWU7MTQyNzI=,a0182e54-72ee-4180-8a02-8b4289a64a04,a1596d5a-f03d-4352-8cf8-64cb1d69dd79,c429d063621d86a8e77b8839137870ad57b7b155ef464c...,2022-07-31 22:42:33,13.624,777.0,Fire TV,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN
48275,view,2022-07-23,NaN,NaN,c5fd51b8-8111-4052-ae79-07bea78dc8c8,c429d063621d86a8e77b8839137870ad57b7b155ef464c...,2022-07-23 17:04:34,NaN,NaN,Fire TV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Reshaping - Bringing it all together

In [27]:
genre_plays = beacons.loc[(beacons['action'] == 'play') & (beacons['percent_stream'] >= 0.05)].groupby(['user_primaryid', 'action_time_frame'])['ActionandAdventure', 'Comedy', 'Documentary',
       'Drama', 'Erotic', 'Horror', 'Indie', 'KidsandFamily', 'Romance',
       'ScienceFictionandFantasy', 'Stand-Up', 'Thriller', 'Vault'].sum().reset_index()
genre_plays

session_bounce_rate = 1 - beacons.loc[(beacons['action'] == 'play')].groupby(['user_primaryid', 'action_time_frame'])['session_id'].nunique() / beacons.groupby(['user_primaryid', 'action_time_frame'])['session_id'].nunique()
session_bounce_rate = session_bounce_rate.reset_index().rename(columns={'session_id':'session_bounce_rate'})

total_sessions = beacons.loc[(beacons['action'] == 'play')].groupby(['user_primaryid', 'action_time_frame'])['session_id'].nunique().reset_index().rename(columns={'session_id':'total_sessions'})
total_sessions

total_devices = beacons.loc[(beacons['action'] == 'play')].groupby(['user_primaryid', 'action_time_frame'])['custom_epix_platform'].nunique().reset_index().rename(columns={'custom_epix_platform':'total_devices'})
total_devices

days_accessed = beacons.groupby(['user_primaryid', 'action_time_frame'])['datepartition'].nunique().to_frame().reset_index().rename(columns={'datepartition':'days_accessed'})
days_accessed

watch_comp = beacons.loc[(beacons['action'] == 'play')].groupby(['user_primaryid', 'action_time_frame'])['percent_stream'].mean().reset_index().rename(columns={'percent_stream':'avg_percent_stream'})
watch_comp

mins_watched = beacons.loc[(beacons['action'] == 'play')].groupby(['user_primaryid', 'action_time_frame'])['mins_stream'].sum().reset_index().rename(columns={'mins_stream':'total_mins_stream'})
mins_watched

brands_watched = beacons.loc[(beacons['action'] == 'play')].groupby(['user_primaryid', 'action_time_frame'])['thing_brandid'].nunique().reset_index().rename(columns={'thing_brandid':'total_brands_watched'})
brands_watched

brand_freshness = beacons.loc[(beacons['action'] == 'play')].groupby(['user_primaryid', 'action_time_frame'])['thing_publication_startdate'].max().reset_index()
brand_freshness.loc[(brand_freshness['action_time_frame'] == 'Under 30 Days'), 'age_newest_content'] = round((pd.to_datetime('2022-09-17') - brand_freshness['thing_publication_startdate']) / np.timedelta64(1, 'M'), 0)
brand_freshness.loc[(brand_freshness['action_time_frame'] == '30 - 60 Days'), 'age_newest_content'] = round((pd.to_datetime('2022-08-17') - brand_freshness['thing_publication_startdate']) / np.timedelta64(1, 'M'), 0)
brand_freshness.loc[(brand_freshness['action_time_frame'] == '60+ Days'), 'age_newest_content'] = round((pd.to_datetime('2022-07-17') - brand_freshness['thing_publication_startdate']) / np.timedelta64(1, 'M'), 0)
brand_freshness = brand_freshness.reset_index()[['age_newest_content', 'user_primaryid', 'action_time_frame']]
brand_freshness

recs_used = beacons.groupby(['user_primaryid', 'action_time_frame'])['custom_rule_id'].count().reset_index().rename(columns={'custom_rule_id':'recs_used'})
recs_used

proportion_movies = beacons.loc[(beacons['action'] == 'play') & (beacons['thing_typename'] == 'Movie')].groupby(['user_primaryid', 'action_time_frame'])['mins_stream'].sum() / beacons.loc[(beacons['action'] == 'play') & ((beacons['thing_typename'] == 'Movie') | beacons['thing_typename'] == 'Episode')].groupby(['user_primaryid', 'action_time_frame'])['mins_stream'].sum()
proportion_movies = proportion_movies.reset_index().rename(columns={'mins_stream':'percent_mins_movies'})
proportion_movies

approx_series_completion = beacons.loc[(beacons['action'] == 'play') & (beacons['thing_typename'] == 'Episode')].groupby(['user_primaryid', 'action_time_frame'])['thing_seasonnumber'].sum() / beacons.loc[(beacons['action'] == 'play') & (beacons['thing_typename'] == 'Episode')].groupby(['user_primaryid', 'action_time_frame'])['number_of_seasons'].sum()
approx_series_completion = approx_series_completion.reset_index().rename(columns={0:'approx_series_completion'})
approx_series_completion

sd_percent = beacons.groupby(['user_primaryid', 'action_time_frame'])['sd_play'].sum() / beacons.loc[(beacons['action'] == 'play') & (beacons['thing_typename'] == 'Episode')].groupby(['user_primaryid', 'action_time_frame'])['play'].sum()
sd_percent = sd_percent.reset_index().rename(columns={0:'sd_percent'})
sd_percent

master_beacons = pd.merge(genre_plays, session_bounce_rate, on=['user_primaryid', 'action_time_frame'], how='outer')
master_beacons = pd.merge(master_beacons, total_sessions, on=['user_primaryid', 'action_time_frame'], how='outer')
master_beacons = pd.merge(master_beacons, total_devices, on=['user_primaryid', 'action_time_frame'], how='outer')
master_beacons = pd.merge(master_beacons, days_accessed, on=['user_primaryid', 'action_time_frame'], how='outer')
master_beacons = pd.merge(master_beacons, watch_comp, on=['user_primaryid', 'action_time_frame'], how='outer')
master_beacons = pd.merge(master_beacons, mins_watched, on=['user_primaryid', 'action_time_frame'], how='outer')
master_beacons = pd.merge(master_beacons, brands_watched, on=['user_primaryid', 'action_time_frame'], how='outer')
master_beacons = pd.merge(master_beacons, brand_freshness, on=['user_primaryid', 'action_time_frame'], how='outer')
master_beacons = pd.merge(master_beacons, recs_used, on=['user_primaryid', 'action_time_frame'], how='outer')
master_beacons = pd.merge(master_beacons, proportion_movies, on=['user_primaryid', 'action_time_frame'], how='outer')
master_beacons = pd.merge(master_beacons, approx_series_completion, on=['user_primaryid', 'action_time_frame'], how='outer')
master_beacons = pd.merge(master_beacons, sd_percent, on=['user_primaryid', 'action_time_frame'], how='outer')

master_beacons.head(20)

/var/folders/72/hqzcd6_j53s1pz09bwht8gmw0000gp/T/ipykernel_92556/3281175723.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  genre_plays = beacons.loc[(beacons['action'] == 'play') & (beacons['percent_stream'] >= 0.05)].groupby(['user_primaryid', 'action_time_frame'])['ActionandAdventure', 'Comedy', 'Documentary',


,user_primaryid,action_time_frame,ActionandAdventure,Comedy,Documentary,Drama,Erotic,Horror,Indie,KidsandFamily,...,total_devices,days_accessed,avg_percent_stream,total_mins_stream,total_brands_watched,age_newest_content,recs_used,percent_mins_movies,approx_series_completion,sd_percent
0,000cfeaf3254e33d89575ec748f2d20065264724b16eee...,30 - 60 Days,7.0,1.0,7.0,3.0,0.0,0.0,0.0,0.0,...,1.0,11,0.122762,344.3,11.0,-1.0,14,NaN,0.636364,0.000000
1,000cfeaf3254e33d89575ec748f2d20065264724b16eee...,60+ Days,8.0,12.0,14.0,4.0,0.0,0.0,0.0,0.0,...,1.0,20,0.110074,876.5,32.0,1.0,26,NaN,0.492063,0.000000
2,000cfeaf3254e33d89575ec748f2d20065264724b16eee...,Under 30 Days,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,6,0.102100,16.7,2.0,25.0,4,NaN,1.000000,0.000000
3,00221569e1b7c27a87939ef94f9b759eea9238e5f3f741...,60+ Days,1.0,0.0,0.0,1.0,0.0,4.0,0.0,0.0,...,1.0,2,0.639400,176.0,2.0,7.0,1,NaN,1.000000,0.000000
4,00221569e1b7c27a87939ef94f9b759eea9238e5f3f741...,Under 30 Days,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,1.0,1,0.873920,219.0,1.0,12.0,0,NaN,1.000000,0.000000
5,00507efc16b56413ce96877909d6434eda0644dc8fa56c...,Under 30 Days,2.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,1.0,3,0.499650,392.4,6.0,0.0,7,NaN,0.947368,0.000000
6,00aa3ec85d8cf0ee3f7bccba4b817ee0ee802586693d05...,Under 30 Days,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,2,0.910878,400.6,1.0,14.0,0,NaN,0.592593,0.000000
7,00d822d05320edaa59c4d14815b30c693ac2951b1a2d59...,Under 30 Days,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,5,0.401132,625.0,2.0,9.0,3,NaN,0.500000,0.000000
8,00fa3bf638e92141888e050204ddbe65b0c8e16b24934a...,30 - 60 Days,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,4,0.341767,50.3,1.0,14.0,0,NaN,0.555556,0.000000
9,00fa3bf638e92141888e050204ddbe65b0c8e16b24934a...,60+ Days,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1,0.660433,96.7,1.0,13.0,0,NaN,0.666667,0.000000


In [28]:
master_beacons.columns

Index(['user_primaryid', 'action_time_frame', 'ActionandAdventure', 'Comedy',
       'Documentary', 'Drama', 'Erotic', 'Horror', 'Indie', 'KidsandFamily',
       'Romance', 'ScienceFictionandFantasy', 'Stand-Up', 'Thriller', 'Vault',
       'session_bounce_rate', 'total_sessions', 'total_devices',
       'days_accessed', 'avg_percent_stream', 'total_mins_stream',
       'total_brands_watched', 'age_newest_content', 'recs_used',
       'percent_mins_movies', 'approx_series_completion', 'sd_percent'],
      dtype='object')

In [29]:
d = {'user_primaryid': master_beacons['user_primaryid']}
pivot_beacons = pd.DataFrame(data=d).drop_duplicates()
print(len(pivot_beacons))
for row, user in enumerate(master_beacons['user_primaryid']):
    for row in master_beacons.columns.delete(0).delete(0):
        pivot_beacons.loc[pivot_beacons['user_primaryid'] == user, row + master_beacons.loc[master_beacons['user_primaryid'] == user]['action_time_frame']] = master_beacons.loc[master_beacons['user_primaryid'] == user][row]        # print(pivot_beacons.loc[pivot_beacons['user_primaryid'] == user][row + master_beacons.loc[master_beacons['user_primaryid'] == user]['action_time_frame']])

print(pivot_beacons.head(20))
pivot_beacons.columns

776
                                       user_primaryid  \
0   000cfeaf3254e33d89575ec748f2d20065264724b16eee...   
3   00221569e1b7c27a87939ef94f9b759eea9238e5f3f741...   
5   00507efc16b56413ce96877909d6434eda0644dc8fa56c...   
6   00aa3ec85d8cf0ee3f7bccba4b817ee0ee802586693d05...   
7   00d822d05320edaa59c4d14815b30c693ac2951b1a2d59...   
8   00fa3bf638e92141888e050204ddbe65b0c8e16b24934a...   
11  01e1eba2619085ba3f241bae1e1522053709600eefac93...   
13  024191558fb2777b87ea22d492c0fa2c8e3c20a4034c15...   
16  026ad40696149d2460829b4978fe168807046be4b917c1...   
17  02d3cc719c03c4a1bb5501b71931451c5610cb4191a1a7...   
20  03087bdfbb825c17e1a368616a5466048f9a5c95b532e1...   
21  04e8e0334e94b42432e7f04ce75783084925ac02cbb525...   
22  055085ce02d240aa509e8cdf86c2b6d0823eec3e862385...   
25  058855b15a41d116c57550ba9e294c52cd32be468fd782...   
27  05f67bc5978f2386d3d4661724158153d1c60307db2b54...   
30  078658374b611167a3b8782ca7d50b8eb4a38efd955a73...   
32  08d5174aadb249c4a53563c

Index(['user_primaryid', 'ActionandAdventure30 - 60 Days',
       'ActionandAdventure60+ Days', 'ActionandAdventureUnder 30 Days',
       'Comedy30 - 60 Days', 'Comedy60+ Days', 'ComedyUnder 30 Days',
       'Documentary30 - 60 Days', 'Documentary60+ Days',
       'DocumentaryUnder 30 Days', 'Drama30 - 60 Days', 'Drama60+ Days',
       'DramaUnder 30 Days', 'Erotic30 - 60 Days', 'Erotic60+ Days',
       'EroticUnder 30 Days', 'Horror30 - 60 Days', 'Horror60+ Days',
       'HorrorUnder 30 Days', 'Indie30 - 60 Days', 'Indie60+ Days',
       'IndieUnder 30 Days', 'KidsandFamily30 - 60 Days',
       'KidsandFamily60+ Days', 'KidsandFamilyUnder 30 Days',
       'Romance30 - 60 Days', 'Romance60+ Days', 'RomanceUnder 30 Days',
       'ScienceFictionandFantasy30 - 60 Days',
       'ScienceFictionandFantasy60+ Days',
       'ScienceFictionandFantasyUnder 30 Days', 'Stand-Up30 - 60 Days',
       'Stand-Up60+ Days', 'Stand-UpUnder 30 Days', 'Thriller30 - 60 Days',
       'Thriller60+ Days', 'Thr

In [30]:
#  User Stats
stats = pd.merge(primary_time_band, primary_algorithm, on=['user_primaryid'], how='outer')
stats = pd.merge(stats, primary_rail, on=['user_primaryid'], how='outer')
stats = pd.merge(stats, primary_device, on=['user_primaryid'], how='outer')
 
master_beacons = pd.merge(pivot_beacons, stats, on=['user_primaryid'], how='outer')
master_beacons.head(20)

,user_primaryid,ActionandAdventure30 - 60 Days,ActionandAdventure60+ Days,ActionandAdventureUnder 30 Days,Comedy30 - 60 Days,Comedy60+ Days,ComedyUnder 30 Days,Documentary30 - 60 Days,Documentary60+ Days,DocumentaryUnder 30 Days,...,approx_series_completion30 - 60 Days,approx_series_completion60+ Days,approx_series_completionUnder 30 Days,sd_percent30 - 60 Days,sd_percent60+ Days,sd_percentUnder 30 Days,favourite_time,favourite_algorithm,favourite_rail,most_used_device
0,000cfeaf3254e33d89575ec748f2d20065264724b16eee...,7.0,7.0,7.0,1.0,1.0,1.0,7.0,7.0,7.0,...,0.636364,0.636364,0.636364,0.0,0.0,0.0,Night,064bbfeb-d46d-4731-8569-aff8c1981293,Home,Apple TV
1,00221569e1b7c27a87939ef94f9b759eea9238e5f3f741...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,1.000000,1.000000,0.0,0.0,0.0,Night,064bbfeb-d46d-4731-8569-aff8c1981293,Home,Roku
2,00507efc16b56413ce96877909d6434eda0644dc8fa56c...,NaN,NaN,2.0,NaN,NaN,0.0,NaN,NaN,1.0,...,NaN,NaN,0.947368,NaN,NaN,0.0,Night,064bbfeb-d46d-4731-8569-aff8c1981293,Home,Fire TV
3,00aa3ec85d8cf0ee3f7bccba4b817ee0ee802586693d05...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,...,NaN,NaN,0.592593,NaN,NaN,0.0,Night,NaN,Home,Fire TV
4,00d822d05320edaa59c4d14815b30c693ac2951b1a2d59...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.500000,0.500000,0.500000,0.0,0.0,0.0,Night,064bbfeb-d46d-4731-8569-aff8c1981293,Home,Apple TV
5,00fa3bf638e92141888e050204ddbe65b0c8e16b24934a...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.555556,0.555556,0.555556,0.0,0.0,0.0,Evening,76dd5146-c45c-464d-92dd-2c9fbfb1ee5d,Home,Apple TV
6,01e1eba2619085ba3f241bae1e1522053709600eefac93...,2.0,NaN,2.0,0.0,NaN,0.0,24.0,NaN,24.0,...,0.601307,NaN,0.601307,0.0,NaN,0.0,Morning,e85f8b98-8175-43d7-847a-7b1140e1343e,Home,Roku
7,024191558fb2777b87ea22d492c0fa2c8e3c20a4034c15...,7.0,7.0,7.0,13.0,13.0,13.0,2.0,2.0,2.0,...,0.600000,0.600000,0.600000,1.0,1.0,1.0,Early_Morning,a0182e54-72ee-4180-8a02-8b4289a64a04,Home,Roku
8,026ad40696149d2460829b4978fe168807046be4b917c1...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,...,NaN,NaN,0.809524,NaN,NaN,0.0,Night,NaN,Home,Apple Tablet
9,02d3cc719c03c4a1bb5501b71931451c5610cb4191a1a7...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Night,a0182e54-72ee-4180-8a02-8b4289a64a04,Home,Apple TV


##### Data Cleansing

In [31]:
len(master_beacons)

master_beacons.columns

Index(['user_primaryid', 'ActionandAdventure30 - 60 Days',
       'ActionandAdventure60+ Days', 'ActionandAdventureUnder 30 Days',
       'Comedy30 - 60 Days', 'Comedy60+ Days', 'ComedyUnder 30 Days',
       'Documentary30 - 60 Days', 'Documentary60+ Days',
       'DocumentaryUnder 30 Days', 'Drama30 - 60 Days', 'Drama60+ Days',
       'DramaUnder 30 Days', 'Erotic30 - 60 Days', 'Erotic60+ Days',
       'EroticUnder 30 Days', 'Horror30 - 60 Days', 'Horror60+ Days',
       'HorrorUnder 30 Days', 'Indie30 - 60 Days', 'Indie60+ Days',
       'IndieUnder 30 Days', 'KidsandFamily30 - 60 Days',
       'KidsandFamily60+ Days', 'KidsandFamilyUnder 30 Days',
       'Romance30 - 60 Days', 'Romance60+ Days', 'RomanceUnder 30 Days',
       'ScienceFictionandFantasy30 - 60 Days',
       'ScienceFictionandFantasy60+ Days',
       'ScienceFictionandFantasyUnder 30 Days', 'Stand-Up30 - 60 Days',
       'Stand-Up60+ Days', 'Stand-UpUnder 30 Days', 'Thriller30 - 60 Days',
       'Thriller60+ Days', 'Thr

In [32]:
master_beacons.isnull().sum().head(20)

user_primaryid                       0
ActionandAdventure30 - 60 Days     473
ActionandAdventure60+ Days         481
ActionandAdventureUnder 30 Days    481
Comedy30 - 60 Days                 473
Comedy60+ Days                     481
ComedyUnder 30 Days                481
Documentary30 - 60 Days            473
Documentary60+ Days                481
DocumentaryUnder 30 Days           481
Drama30 - 60 Days                  473
Drama60+ Days                      481
DramaUnder 30 Days                 481
Erotic30 - 60 Days                 473
Erotic60+ Days                     481
EroticUnder 30 Days                481
Horror30 - 60 Days                 473
Horror60+ Days                     481
HorrorUnder 30 Days                481
Indie30 - 60 Days                  473
dtype: int64

In [33]:
master_beacons[['ActionandAdventure60+ Days', 'Comedy60+ Days', 'Documentary60+ Days',
              'Drama60+ Days', 'Erotic60+ Days', 'Horror60+ Days', 'Indie60+ Days',
              'KidsandFamily60+ Days', 'Romance60+ Days', 'ScienceFictionandFantasy60+ Days',
              'Stand-Up60+ Days', 'Thriller60+ Days', 'Vault60+ Days', 'session_bounce_rate60+ Days', 
              'total_sessions60+ Days', 'total_devices60+ Days', 'days_accessed60+ Days', 'avg_percent_stream60+ Days',
              'total_mins_stream60+ Days', 'total_brands_watched60+ Days', 'age_newest_content60+ Days',
              'percent_mins_movies60+ Days','approx_series_completion60+ Days', 'sd_percent60+ Days', 'ActionandAdventure30 - 60 Days',
              'ActionandAdventureUnder 30 Days', 'Comedy30 - 60 Days',
              'ComedyUnder 30 Days', 'Documentary30 - 60 Days',
              'DocumentaryUnder 30 Days', 'Drama30 - 60 Days', 'DramaUnder 30 Days',
              'Erotic30 - 60 Days', 'EroticUnder 30 Days', 'Horror30 - 60 Days',
              'HorrorUnder 30 Days', 'Indie30 - 60 Days', 'IndieUnder 30 Days',
              'KidsandFamily30 - 60 Days', 'KidsandFamilyUnder 30 Days',
              'Romance30 - 60 Days', 'RomanceUnder 30 Days',
              'ScienceFictionandFantasy30 - 60 Days',
              'ScienceFictionandFantasyUnder 30 Days', 'Stand-Up30 - 60 Days',
              'Stand-UpUnder 30 Days', 'Thriller30 - 60 Days',
              'ThrillerUnder 30 Days', 'Vault30 - 60 Days', 'VaultUnder 30 Days',
              'session_bounce_rate30 - 60 Days', 'session_bounce_rateUnder 30 Days',
              'total_sessions30 - 60 Days', 'total_sessionsUnder 30 Days',
              'total_devices30 - 60 Days', 'total_devicesUnder 30 Days',
              'days_accessed30 - 60 Days', 'days_accessedUnder 30 Days', 'recs_used60+ Days',
              'avg_percent_stream30 - 60 Days', 'avg_percent_streamUnder 30 Days',
              'total_mins_stream30 - 60 Days', 'total_mins_streamUnder 30 Days',
              'total_brands_watched30 - 60 Days', 'total_brands_watchedUnder 30 Days',
              'age_newest_content30 - 60 Days', 'age_newest_contentUnder 30 Days',
              'recs_used30 - 60 Days', 'recs_usedUnder 30 Days',
              'percent_mins_movies30 - 60 Days', 'percent_mins_moviesUnder 30 Days',
              'approx_series_completion30 - 60 Days', 'approx_series_completionUnder 30 Days', 'sd_percent30 - 60 Days',
              'sd_percentUnder 30 Days']] = master_beacons[['ActionandAdventure60+ Days', 'Comedy60+ Days', 'Documentary60+ Days',
              'Drama60+ Days', 'Erotic60+ Days', 'Horror60+ Days', 'Indie60+ Days',
              'KidsandFamily60+ Days', 'Romance60+ Days', 'ScienceFictionandFantasy60+ Days',
              'Stand-Up60+ Days', 'Thriller60+ Days', 'Vault60+ Days', 'session_bounce_rate60+ Days', 
              'total_sessions60+ Days', 'total_devices60+ Days', 'days_accessed60+ Days', 'avg_percent_stream60+ Days',
              'total_mins_stream60+ Days', 'total_brands_watched60+ Days', 'age_newest_content60+ Days',
              'percent_mins_movies60+ Days','approx_series_completion60+ Days', 'sd_percent60+ Days', 'ActionandAdventure30 - 60 Days',
              'ActionandAdventureUnder 30 Days', 'Comedy30 - 60 Days',
              'ComedyUnder 30 Days', 'Documentary30 - 60 Days',
              'DocumentaryUnder 30 Days', 'Drama30 - 60 Days', 'DramaUnder 30 Days',
              'Erotic30 - 60 Days', 'EroticUnder 30 Days', 'Horror30 - 60 Days',
              'HorrorUnder 30 Days', 'Indie30 - 60 Days', 'IndieUnder 30 Days',
              'KidsandFamily30 - 60 Days', 'KidsandFamilyUnder 30 Days',
              'Romance30 - 60 Days', 'RomanceUnder 30 Days',
              'ScienceFictionandFantasy30 - 60 Days',
              'ScienceFictionandFantasyUnder 30 Days', 'Stand-Up30 - 60 Days',
              'Stand-UpUnder 30 Days', 'Thriller30 - 60 Days', 'recs_used60+ Days', 
              'ThrillerUnder 30 Days', 'Vault30 - 60 Days', 'VaultUnder 30 Days',
              'session_bounce_rate30 - 60 Days', 'session_bounce_rateUnder 30 Days',
              'total_sessions30 - 60 Days', 'total_sessionsUnder 30 Days',
              'total_devices30 - 60 Days', 'total_devicesUnder 30 Days',
              'days_accessed30 - 60 Days', 'days_accessedUnder 30 Days',
              'avg_percent_stream30 - 60 Days', 'avg_percent_streamUnder 30 Days',
              'total_mins_stream30 - 60 Days', 'total_mins_streamUnder 30 Days',
              'total_brands_watched30 - 60 Days', 'total_brands_watchedUnder 30 Days',
              'age_newest_content30 - 60 Days', 'age_newest_contentUnder 30 Days',
              'recs_used30 - 60 Days', 'recs_usedUnder 30 Days',
              'percent_mins_movies30 - 60 Days', 'percent_mins_moviesUnder 30 Days',
              'approx_series_completion30 - 60 Days', 'approx_series_completionUnder 30 Days', 'sd_percent30 - 60 Days',
              'sd_percentUnder 30 Days']].fillna(0)

master_beacons['favourite_rail'] = master_beacons['favourite_rail'].fillna('Home')
master_beacons['favourite_algorithm'] = master_beacons['favourite_algorithm'].fillna('NA')
master_beacons['most_used_device'] = master_beacons['most_used_device'].fillna('Unknown')


master_beacons.head(20)

,user_primaryid,ActionandAdventure30 - 60 Days,ActionandAdventure60+ Days,ActionandAdventureUnder 30 Days,Comedy30 - 60 Days,Comedy60+ Days,ComedyUnder 30 Days,Documentary30 - 60 Days,Documentary60+ Days,DocumentaryUnder 30 Days,...,approx_series_completion30 - 60 Days,approx_series_completion60+ Days,approx_series_completionUnder 30 Days,sd_percent30 - 60 Days,sd_percent60+ Days,sd_percentUnder 30 Days,favourite_time,favourite_algorithm,favourite_rail,most_used_device
0,000cfeaf3254e33d89575ec748f2d20065264724b16eee...,7.0,7.0,7.0,1.0,1.0,1.0,7.0,7.0,7.0,...,0.636364,0.636364,0.636364,0.0,0.0,0.0,Night,064bbfeb-d46d-4731-8569-aff8c1981293,Home,Apple TV
1,00221569e1b7c27a87939ef94f9b759eea9238e5f3f741...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,1.000000,1.000000,0.0,0.0,0.0,Night,064bbfeb-d46d-4731-8569-aff8c1981293,Home,Roku
2,00507efc16b56413ce96877909d6434eda0644dc8fa56c...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.000000,0.000000,0.947368,0.0,0.0,0.0,Night,064bbfeb-d46d-4731-8569-aff8c1981293,Home,Fire TV
3,00aa3ec85d8cf0ee3f7bccba4b817ee0ee802586693d05...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.592593,0.0,0.0,0.0,Night,NA,Home,Fire TV
4,00d822d05320edaa59c4d14815b30c693ac2951b1a2d59...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.500000,0.500000,0.500000,0.0,0.0,0.0,Night,064bbfeb-d46d-4731-8569-aff8c1981293,Home,Apple TV
5,00fa3bf638e92141888e050204ddbe65b0c8e16b24934a...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.555556,0.555556,0.555556,0.0,0.0,0.0,Evening,76dd5146-c45c-464d-92dd-2c9fbfb1ee5d,Home,Apple TV
6,01e1eba2619085ba3f241bae1e1522053709600eefac93...,2.0,0.0,2.0,0.0,0.0,0.0,24.0,0.0,24.0,...,0.601307,0.000000,0.601307,0.0,0.0,0.0,Morning,e85f8b98-8175-43d7-847a-7b1140e1343e,Home,Roku
7,024191558fb2777b87ea22d492c0fa2c8e3c20a4034c15...,7.0,7.0,7.0,13.0,13.0,13.0,2.0,2.0,2.0,...,0.600000,0.600000,0.600000,1.0,1.0,1.0,Early_Morning,a0182e54-72ee-4180-8a02-8b4289a64a04,Home,Roku
8,026ad40696149d2460829b4978fe168807046be4b917c1...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.809524,0.0,0.0,0.0,Night,NA,Home,Apple Tablet
9,02d3cc719c03c4a1bb5501b71931451c5610cb4191a1a7...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,Night,a0182e54-72ee-4180-8a02-8b4289a64a04,Home,Apple TV


# Checking 'Favourite' Stats for potentical slimming down
- favourite_algorithm is fine as is (only 6 variables and all different
- favourite_rail can be slimmed down e.g. Movie DP and Show DP --> DP as the same
- most_used_device can be slimmed down into device categories



In [34]:
master_beacons['favourite_algorithm'].value_counts()
master_beacons['favourite_rail'].value_counts()

master_beacons.loc[(master_beacons['favourite_rail'] == 'Movie DP') | (master_beacons['favourite_rail'] == 'Show DP'), 'f_rail_condensed'] = 'DP'
master_beacons.loc[(master_beacons['favourite_rail'] == 'Search') | (master_beacons['favourite_rail'] == 'Search Results') | (master_beacons['favourite_rail'] == 'Queue') | (master_beacons['favourite_rail'] == 'Browse') | (master_beacons['favourite_rail'] == 'History'), 'f_rail_condensed'] = 'User_Led'
master_beacons.loc[(master_beacons['favourite_rail'] == 'Movies') | (master_beacons['favourite_rail'] == 'Originals'), 'f_rail_condensed'] = 'Movies/Orginals'
master_beacons['f_rail_condensed'] = master_beacons['f_rail_condensed'].fillna(master_beacons['favourite_rail'])
master_beacons['f_rail_condensed'].value_counts()

master_beacons['most_used_device'].value_counts()

master_beacons.loc[(master_beacons['most_used_device'] == 'Roku') | (master_beacons['most_used_device'] == 'Fire TV') | (master_beacons['most_used_device'] == 'Apple TV') | (master_beacons['most_used_device'] == 'Android TV') | (master_beacons['most_used_device'] == 'Samsung TV'), 'device_condensed'] = 'TV_Device'
master_beacons.loc[(master_beacons['most_used_device'] == 'Apple Mobile') | (master_beacons['most_used_device'] == 'Android Mobile'), 'device_condensed'] = 'Mobile'
master_beacons.loc[(master_beacons['most_used_device'] == 'Apple Tablet') | (master_beacons['most_used_device'] == 'Android Tablet') | (master_beacons['most_used_device'] == 'Kindle Fire'), 'device_condensed'] = 'Tablet'
master_beacons['device_condensed'] = master_beacons['device_condensed'].fillna(master_beacons['most_used_device'])

master_beacons['device_condensed'].value_counts()

# master_beacons


TV_Device    521
Mobile       148
Tablet        69
Web           34
Unknown        4
Name: device_condensed, dtype: int64

In [35]:
# Creating Dummies
time_dummies = pd.get_dummies(master_beacons['favourite_time'], prefix = 'time')
master_beacons = pd.concat([master_beacons, time_dummies], axis=1)

alg_dummies = pd.get_dummies(master_beacons['favourite_algorithm'], prefix = 'alg')
master_beacons = pd.concat([master_beacons, alg_dummies], axis=1)

rail_dummies = pd.get_dummies(master_beacons['f_rail_condensed'], prefix = 'rail')
master_beacons = pd.concat([master_beacons, rail_dummies], axis=1)

device_dummies = pd.get_dummies(master_beacons['device_condensed'], prefix = 'device')
master_beacons = pd.concat([master_beacons, device_dummies], axis=1)

time_frame_dummies = pd.get_dummies(master_beacons['favourite_time'], prefix = 'time_frame')
master_beacons = pd.concat([master_beacons, time_frame_dummies], axis=1)

master_beacons.head(20)

,user_primaryid,ActionandAdventure30 - 60 Days,ActionandAdventure60+ Days,ActionandAdventureUnder 30 Days,Comedy30 - 60 Days,Comedy60+ Days,ComedyUnder 30 Days,Documentary30 - 60 Days,Documentary60+ Days,DocumentaryUnder 30 Days,...,device_Mobile,device_TV_Device,device_Tablet,device_Unknown,device_Web,time_frame_Afternoon,time_frame_Early_Morning,time_frame_Evening,time_frame_Morning,time_frame_Night
0,000cfeaf3254e33d89575ec748f2d20065264724b16eee...,7.0,7.0,7.0,1.0,1.0,1.0,7.0,7.0,7.0,...,0,1,0,0,0,0,0,0,0,1
1,00221569e1b7c27a87939ef94f9b759eea9238e5f3f741...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,1
2,00507efc16b56413ce96877909d6434eda0644dc8fa56c...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,1,0,0,0,0,0,0,0,1
3,00aa3ec85d8cf0ee3f7bccba4b817ee0ee802586693d05...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,1
4,00d822d05320edaa59c4d14815b30c693ac2951b1a2d59...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,1
5,00fa3bf638e92141888e050204ddbe65b0c8e16b24934a...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,1,0,0
6,01e1eba2619085ba3f241bae1e1522053709600eefac93...,2.0,0.0,2.0,0.0,0.0,0.0,24.0,0.0,24.0,...,0,1,0,0,0,0,0,0,1,0
7,024191558fb2777b87ea22d492c0fa2c8e3c20a4034c15...,7.0,7.0,7.0,13.0,13.0,13.0,2.0,2.0,2.0,...,0,1,0,0,0,0,1,0,0,0
8,026ad40696149d2460829b4978fe168807046be4b917c1...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,1
9,02d3cc719c03c4a1bb5501b71931451c5610cb4191a1a7...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,1


In [36]:
pd.options.display.max_rows = 4000
pd.options.display.max_seq_items = 4000

for col in master_beacons.columns:
    print("'"+col+"',")

'user_primaryid',
'ActionandAdventure30 - 60 Days',
'ActionandAdventure60+ Days',
'ActionandAdventureUnder 30 Days',
'Comedy30 - 60 Days',
'Comedy60+ Days',
'ComedyUnder 30 Days',
'Documentary30 - 60 Days',
'Documentary60+ Days',
'DocumentaryUnder 30 Days',
'Drama30 - 60 Days',
'Drama60+ Days',
'DramaUnder 30 Days',
'Erotic30 - 60 Days',
'Erotic60+ Days',
'EroticUnder 30 Days',
'Horror30 - 60 Days',
'Horror60+ Days',
'HorrorUnder 30 Days',
'Indie30 - 60 Days',
'Indie60+ Days',
'IndieUnder 30 Days',
'KidsandFamily30 - 60 Days',
'KidsandFamily60+ Days',
'KidsandFamilyUnder 30 Days',
'Romance30 - 60 Days',
'Romance60+ Days',
'RomanceUnder 30 Days',
'ScienceFictionandFantasy30 - 60 Days',
'ScienceFictionandFantasy60+ Days',
'ScienceFictionandFantasyUnder 30 Days',
'Stand-Up30 - 60 Days',
'Stand-Up60+ Days',
'Stand-UpUnder 30 Days',
'Thriller30 - 60 Days',
'Thriller60+ Days',
'ThrillerUnder 30 Days',
'Vault30 - 60 Days',
'Vault60+ Days',
'VaultUnder 30 Days',
'session_bounce_rate30 - 60 Day

##### Cleaning of the User Table
- The user table contains details on whether each user has churned among other stats

In [37]:
user_table = pd.read_csv('/Users/mattcadel/Documents/Python/DSML/user_table.csv')
user_table.head()

,user_primaryid,user_has_churned,custom_epix_billingcycle,custom_epix_billingprovider,custom_epix_productprice,user_had_free_trial,months_first_sign_in
0,5274f3fbe6b716549cee37e99a87b18af61c53c3c72fa4...,1,Monthly,Apple,5.99,0,6+_month_user
1,63da7142fd840e18df5f8255646b36dbcd3abde6edf49b...,1,Monthly,Apple,5.99,0,new_user
2,14e9b974ae1c7bc60d1776dcfd0efba9b0138bee1c918a...,1,NaN,Spectrum,NaN,0,6+_month_user
3,67c1c117cc0a6c3f8a2b0057b08e3315bc4ac2bdf8e557...,1,Monthly,Amazon,5.99,0,6+_month_user
4,7a3353ec0ddddbc202d231e7166a4e8ae0595b800b6d3f...,1,Monthly,Amazon,5.99,0,6+_month_user


In [38]:
user_table.isnull().sum()


user_primaryid                   0
user_has_churned                 0
custom_epix_billingcycle       191
custom_epix_billingprovider      0
custom_epix_productprice       224
user_had_free_trial              0
months_first_sign_in             0
dtype: int64

In [39]:
len(user_table)

776

##### Dealing with null values and feature engineering
- Convert first_sign_in into weeks from period end
- Fill in nulls of annual_rev
- Create feature of billingprovider 
- Create a dummy for whether a user is a monthly payer


In [40]:
first_sign_in_dummies = pd.get_dummies(user_table['months_first_sign_in'], prefix = 'sign_in')
user_table = pd.concat([user_table, first_sign_in_dummies], axis=1)

user_table.loc[user_table['custom_epix_billingcycle'] == 'Semi-annual', 'annual_rev'] = 2 * user_table['custom_epix_productprice'].loc[user_table['custom_epix_billingcycle'] == 'Semi-annual']
user_table.loc[user_table['custom_epix_billingcycle'] == 'Annual', 'annual_rev'] = user_table['custom_epix_productprice'].loc[user_table['custom_epix_billingcycle'] == 'Annual']
user_table.loc[user_table['custom_epix_billingcycle'] == 'Monthly', 'annual_rev'] = 12 * user_table['custom_epix_productprice'].loc[user_table['custom_epix_billingcycle'] == 'Monthly']

user_table['annual_rev'].loc[user_table['custom_epix_billingcycle'] == 'Semi-annual'] = user_table['annual_rev'].loc[user_table['custom_epix_billingcycle'] == 'Semi-annual'].fillna(user_table.loc[user_table['custom_epix_billingcycle'] == 'Semi-annual']['annual_rev'].median())
user_table['annual_rev'].loc[user_table['custom_epix_billingcycle'] == 'Annual'] = user_table['annual_rev'].loc[user_table['custom_epix_billingcycle'] == 'Annual'].fillna(user_table.loc[user_table['custom_epix_billingcycle'] == 'Annual']['annual_rev'].median())
user_table['annual_rev'].loc[user_table['custom_epix_billingcycle'] == 'Monthly'] = user_table['annual_rev'].loc[user_table['custom_epix_billingcycle'] == 'Monthly'].fillna(user_table.loc[user_table['custom_epix_billingcycle'] == 'Monthly']['annual_rev'].median())
user_table['annual_rev'] = user_table['annual_rev'].fillna(user_table['annual_rev'].median())

user_table['custom_epix_billingprovider'].unique() # 20 unique values, therefore will not use thoroughly, will only create dummy for direct EPIX payers
user_table.loc[(user_table['custom_epix_billingprovider'] == 'EPIX'), 'direct_sub'] = 1
user_table.loc[(user_table['custom_epix_billingprovider'] != 'EPIX'), 'direct_sub'] = 0

user_table.loc[(user_table['custom_epix_billingcycle'] == 'Monthly'), 'monthly_billing'] = 1
user_table.loc[(user_table['custom_epix_billingcycle'] != 'Monthly'), 'monthly_billing'] = 0


user_table.head(20)
user_table.isnull().sum()



/var/folders/72/hqzcd6_j53s1pz09bwht8gmw0000gp/T/ipykernel_92556/3799606015.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_table['annual_rev'].loc[user_table['custom_epix_billingcycle'] == 'Semi-annual'] = user_table['annual_rev'].loc[user_table['custom_epix_billingcycle'] == 'Semi-annual'].fillna(user_table.loc[user_table['custom_epix_billingcycle'] == 'Semi-annual']['annual_rev'].median())
/var/folders/72/hqzcd6_j53s1pz09bwht8gmw0000gp/T/ipykernel_92556/3799606015.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_table['annual_rev'].loc[user_table['custom_epix_billingcycle'] == 'Annual'] = user_table

user_primaryid                   0
user_has_churned                 0
custom_epix_billingcycle       191
custom_epix_billingprovider      0
custom_epix_productprice       224
user_had_free_trial              0
months_first_sign_in             0
sign_in_3+_month_user            0
sign_in_6+_month_user            0
sign_in_new_user                 0
annual_rev                       0
direct_sub                       0
monthly_billing                  0
dtype: int64

### Joining the user table to beacons and converting action_time_frame into a usable axis for ML

In [41]:
ml_table = pd.merge(master_beacons, user_table, on=['user_primaryid'], how='outer')


ml_table.head(20)
ml_table.loc[ml_table['user_has_churned'] == 0].isnull().sum()

# 3c632017fadebe1bd16bf85610ccc87bcc8271da88e82f6cc8af7a7e5adcc52b
# 3acca6751a59ea6792c681aaf749f036153347550f72d2cc3e1fb7cd379f8432
# 41f3e0fc951ebe28034ed0b438c1eaab597c72d048407876e9fa894b541ae24a
# 51f3e6d30fec8a385abdbb3cbb0d0be82426689186279ab7bc997932ecd6f704

user_primaryid                                0
ActionandAdventure30 - 60 Days                0
ActionandAdventure60+ Days                    0
ActionandAdventureUnder 30 Days               0
Comedy30 - 60 Days                            0
Comedy60+ Days                                0
ComedyUnder 30 Days                           0
Documentary30 - 60 Days                       0
Documentary60+ Days                           0
DocumentaryUnder 30 Days                      0
Drama30 - 60 Days                             0
Drama60+ Days                                 0
DramaUnder 30 Days                            0
Erotic30 - 60 Days                            0
Erotic60+ Days                                0
EroticUnder 30 Days                           0
Horror30 - 60 Days                            0
Horror60+ Days                                0
HorrorUnder 30 Days                           0
Indie30 - 60 Days                             0
Indie60+ Days                           

### Preparation for ML

##### Firstly I use the linear model

In [42]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt

ml_table = ml_table[['ActionandAdventure30 - 60 Days', 'ActionandAdventure60+ Days', 'ActionandAdventureUnder 30 Days',
                    'Comedy30 - 60 Days', 'Comedy60+ Days', 'ComedyUnder 30 Days', 'Documentary30 - 60 Days',
                    'Documentary60+ Days', 'DocumentaryUnder 30 Days', 'Drama30 - 60 Days', 'Drama60+ Days',
                    'DramaUnder 30 Days', 'Erotic30 - 60 Days', 'Erotic60+ Days', 'EroticUnder 30 Days',
                    'Horror30 - 60 Days', 'Horror60+ Days', 'HorrorUnder 30 Days', 'Indie30 - 60 Days',
                    'Indie60+ Days', 'IndieUnder 30 Days', 'KidsandFamily30 - 60 Days', 'KidsandFamily60+ Days', 
                    'KidsandFamilyUnder 30 Days', 'Romance30 - 60 Days', 'Romance60+ Days', 'RomanceUnder 30 Days',
                    'ScienceFictionandFantasy30 - 60 Days', 'ScienceFictionandFantasy60+ Days', 'ScienceFictionandFantasyUnder 30 Days',
                    'Stand-Up30 - 60 Days', 'Stand-Up60+ Days', 'Stand-UpUnder 30 Days', 'Thriller30 - 60 Days', 'Thriller60+ Days',
                    'ThrillerUnder 30 Days', 'Vault30 - 60 Days', 'Vault60+ Days', 'VaultUnder 30 Days', 'session_bounce_rate30 - 60 Days', 
                    'session_bounce_rate60+ Days','session_bounce_rateUnder 30 Days', 'total_sessions30 - 60 Days', 'total_sessions60+ Days',
                    'total_sessionsUnder 30 Days', 'total_devices30 - 60 Days', 'total_devices60+ Days', 'total_devicesUnder 30 Days', 
                    'days_accessed30 - 60 Days', 'days_accessed60+ Days', 'days_accessedUnder 30 Days', 'avg_percent_stream30 - 60 Days', 
                    'avg_percent_stream60+ Days', 'avg_percent_streamUnder 30 Days', 'total_mins_stream30 - 60 Days', 'total_mins_stream60+ Days', 
                    'total_mins_streamUnder 30 Days', 'total_brands_watched30 - 60 Days', 'total_brands_watched60+ Days', 'total_brands_watchedUnder 30 Days', 
                    'age_newest_content30 - 60 Days', 'age_newest_content60+ Days', 'age_newest_contentUnder 30 Days', 'recs_used30 - 60 Days',
                    'recs_used60+ Days', 'recs_usedUnder 30 Days', 'percent_mins_movies30 - 60 Days', 'percent_mins_movies60+ Days',
                    'percent_mins_moviesUnder 30 Days', 'approx_series_completion30 - 60 Days', 'approx_series_completion60+ Days', 'approx_series_completionUnder 30 Days',
                    'sd_percent30 - 60 Days', 'sd_percent60+ Days', 'sd_percentUnder 30 Days', 'time_Afternoon', 
                     'time_Early_Morning', 'time_Evening', 'user_had_free_trial',
                     'time_Morning', 'time_Night', 'alg_064bbfeb-d46d-4731-8569-aff8c1981293', 'alg_63740562-701f-4391-87da-baf604c28a31', 'alg_76dd5146-c45c-464d-92dd-2c9fbfb1ee5d', 
                     'alg_NA','alg_a0182e54-72ee-4180-8a02-8b4289a64a04', 'alg_e85f8b98-8175-43d7-847a-7b1140e1343e', 'alg_f7cb342f-4dc0-4180-bf47-8e0413a49c8f',
                     'rail_DP', 'rail_Home', 'rail_Movies/Orginals', 'rail_User_Led', 'device_Mobile', 'device_TV_Device', 'device_Tablet', 'device_Unknown',
                     'device_Web', 'time_frame_Afternoon', 'time_frame_Early_Morning', 'time_frame_Evening', 'time_frame_Morning', 'time_frame_Night', 
                     'user_has_churned', 'monthly_billing', 'direct_sub', 'annual_rev', 'sign_in_new_user', 'sign_in_6+_month_user', 'sign_in_3+_month_user']]
# ml_table = ml_table.dropna()

X = ml_table[['ActionandAdventure30 - 60 Days', 'ActionandAdventure60+ Days', 'ActionandAdventureUnder 30 Days',
                    'Comedy30 - 60 Days', 'Comedy60+ Days', 'ComedyUnder 30 Days', 'Documentary30 - 60 Days',
                    'Documentary60+ Days', 'DocumentaryUnder 30 Days', 'Drama30 - 60 Days', 'Drama60+ Days',
                    'DramaUnder 30 Days', 'Erotic30 - 60 Days', 'Erotic60+ Days', 'EroticUnder 30 Days',
                    'Horror30 - 60 Days', 'Horror60+ Days', 'HorrorUnder 30 Days', 'Indie30 - 60 Days',
                    'Indie60+ Days', 'IndieUnder 30 Days', 'KidsandFamily30 - 60 Days', 'KidsandFamily60+ Days', 
                    'KidsandFamilyUnder 30 Days', 'Romance30 - 60 Days', 'Romance60+ Days', 'RomanceUnder 30 Days',
                    'ScienceFictionandFantasy30 - 60 Days', 'ScienceFictionandFantasy60+ Days', 'ScienceFictionandFantasyUnder 30 Days',
                    'Stand-Up30 - 60 Days', 'Stand-Up60+ Days', 'Stand-UpUnder 30 Days', 'Thriller30 - 60 Days', 'Thriller60+ Days',
                    'ThrillerUnder 30 Days', 'Vault30 - 60 Days', 'Vault60+ Days', 'VaultUnder 30 Days', 'session_bounce_rate30 - 60 Days', 
                    'session_bounce_rate60+ Days','session_bounce_rateUnder 30 Days', 'total_sessions30 - 60 Days', 'total_sessions60+ Days',
                    'total_sessionsUnder 30 Days', 'total_devices30 - 60 Days', 'total_devices60+ Days', 'total_devicesUnder 30 Days', 
                    'days_accessed30 - 60 Days', 'days_accessed60+ Days', 'days_accessedUnder 30 Days', 'avg_percent_stream30 - 60 Days', 
                    'avg_percent_stream60+ Days', 'avg_percent_streamUnder 30 Days', 'total_mins_stream30 - 60 Days', 'total_mins_stream60+ Days', 
                    'total_mins_streamUnder 30 Days', 'total_brands_watched30 - 60 Days', 'total_brands_watched60+ Days', 'total_brands_watchedUnder 30 Days', 
                    'age_newest_content30 - 60 Days', 'age_newest_content60+ Days', 'age_newest_contentUnder 30 Days', 'recs_used30 - 60 Days',
                    'recs_used60+ Days', 'recs_usedUnder 30 Days', 'percent_mins_movies30 - 60 Days', 'percent_mins_movies60+ Days',
                    'percent_mins_moviesUnder 30 Days', 'approx_series_completion30 - 60 Days', 'approx_series_completion60+ Days', 'approx_series_completionUnder 30 Days',
                    'sd_percent30 - 60 Days', 'sd_percent60+ Days', 'sd_percentUnder 30 Days', 'time_Afternoon', 
                     'time_Early_Morning', 'time_Evening', 'user_had_free_trial',
                     'time_Morning', 'time_Night', 'alg_064bbfeb-d46d-4731-8569-aff8c1981293', 'alg_63740562-701f-4391-87da-baf604c28a31', 'alg_76dd5146-c45c-464d-92dd-2c9fbfb1ee5d', 
                     'alg_NA','alg_a0182e54-72ee-4180-8a02-8b4289a64a04', 'alg_e85f8b98-8175-43d7-847a-7b1140e1343e', 'alg_f7cb342f-4dc0-4180-bf47-8e0413a49c8f',
                     'rail_DP', 'rail_Home', 'rail_Movies/Orginals', 'rail_User_Led', 'device_Mobile', 'device_TV_Device', 'device_Tablet', 'device_Unknown',
                     'device_Web', 'time_frame_Afternoon', 'time_frame_Early_Morning', 'time_frame_Evening', 'time_frame_Morning', 'time_frame_Night',
                      'monthly_billing', 'direct_sub', 'annual_rev', 'sign_in_new_user', 'sign_in_6+_month_user', 'sign_in_3+_month_user']]

y = ml_table['user_has_churned']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("X_train shape: {}, X_test shape: {}, y_train shape: {}, y_test shape {}".format(X_train.shape, X_test.shape, y_train.shape, y_test.shape))
print(y.unique())

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score, accuracy_score

# X = df[['Pclass', 'Age', 'SibSp', 'Parch', 'female']]
# y = df.Survived

model = LogisticRegression()
model.fit(X_train, y_train)
predictions_train = model.predict(X_train)
roc_auc = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])
print("ROC-AUC:",roc_auc)

print("MSE:",mean_squared_error(y_train, predictions_train))

train_accuracy = accuracy_score(y_train, model.predict(X_train))
print("Train accuracy:", train_accuracy)
test_accuracy = accuracy_score(y_test, model.predict(X_test))
print("Test accuracy:" , test_accuracy)

coef = model.coef_[0]
for n, i in enumerate(coef):
    print('Coefficient for feature - ', X.columns[n], '\n =', i)

print(predictions_train)

X_train shape: (620, 108), X_test shape: (156, 108), y_train shape: (620,), y_test shape (156,)
[0 1]
ROC-AUC: 0.8195827900912647
MSE: 0.2435483870967742
Train accuracy: 0.7564516129032258
Test accuracy: 0.6923076923076923
Coefficient for feature -  ActionandAdventure30 - 60 Days 
 = 0.041841448945281925
Coefficient for feature -  ActionandAdventure60+ Days 
 = -0.04597601031604254
Coefficient for feature -  ActionandAdventureUnder 30 Days 
 = -0.026261849639501816
Coefficient for feature -  Comedy30 - 60 Days 
 = -0.03552334887108169
Coefficient for feature -  Comedy60+ Days 
 = -0.10898538733231145
Coefficient for feature -  ComedyUnder 30 Days 
 = -0.11583229984914298
Coefficient for feature -  Documentary30 - 60 Days 
 = -0.22556550093492012
Coefficient for feature -  Documentary60+ Days 
 = -0.07603211299350086
Coefficient for feature -  DocumentaryUnder 30 Days 
 = -0.15666271444945332
Coefficient for feature -  Drama30 - 60 Days 
 = 0.0007221042935082131
Coefficient for feature 

/Users/mattcadel/Documents/Python/DSML/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


##### And now for some experimentation with a NN
##### 1 - Experimenting with solvers - adam is best


In [52]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(random_state=1, max_iter=300, hidden_layer_sizes= (10,5), activation = 'relu', solver = 'adam')
model.fit(X_train, y_train)

predictions_train = model.predict(X_train) #  can use predict_proba to output probabilities
roc_auc = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])
print("ROC-AUC:",roc_auc)

print("MSE:",mean_squared_error(y_train, predictions_train))

train_accuracy = accuracy_score(y_train, model.predict(X_train))
print("Train accuracy:", train_accuracy)
test_accuracy = accuracy_score(y_test, model.predict(X_test))
print("Test accuracy:" , test_accuracy)

print(predictions_train)

[[0.38983599 0.61016401]
 [0.50211686 0.49788314]
 [0.89733599 0.10266401]
 ...
 [0.03520367 0.96479633]
 [0.43053064 0.56946936]
 [0.67032392 0.32967608]]


In [44]:
model = MLPClassifier(random_state=1, max_iter=300, hidden_layer_sizes= (10,5), activation = 'relu', solver = 'sgd')
model.fit(X_train, y_train)

predictions_train = model.predict(X_train)
roc_auc = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])
print("ROC-AUC:",roc_auc)

print("MSE:",mean_squared_error(y_train, predictions_train))

train_accuracy = accuracy_score(y_train, model.predict(X_train))
print("Train accuracy:", train_accuracy)
test_accuracy = accuracy_score(y_test, model.predict(X_test))
print("Test accuracy:" , test_accuracy)

ROC-AUC: 0.5246883963494131
MSE: 0.5161290322580645
Train accuracy: 0.4838709677419355
Test accuracy: 0.5256410256410257


In [45]:
model = MLPClassifier(random_state=1, max_iter=300, hidden_layer_sizes= (10,5), activation = 'relu', solver = 'lbfgs')
model.fit(X_train, y_train)

predictions_train = model.predict(X_train)
roc_auc = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])
print("ROC-AUC:",roc_auc)

print("MSE:",mean_squared_error(y_train, predictions_train))

train_accuracy = accuracy_score(y_train, model.predict(X_train))
print("Train accuracy:", train_accuracy)
test_accuracy = accuracy_score(y_test, model.predict(X_test))
print("Test accuracy:" , test_accuracy)

ROC-AUC: 0.8622998696219035
MSE: 0.20483870967741935
Train accuracy: 0.7951612903225806
Test accuracy: 0.7564102564102564


/Users/mattcadel/Documents/Python/DSML/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


##### 2 - Experimenting with activation - logistic is best

In [46]:
model = MLPClassifier(random_state=1, max_iter=300, hidden_layer_sizes= (10,5), activation = 'identity', solver = 'adam')
model.fit(X_train, y_train)

predictions_train = model.predict(X_train)
roc_auc = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])
print("ROC-AUC:",roc_auc)

print("MSE:",mean_squared_error(y_train, predictions_train))

train_accuracy = accuracy_score(y_train, model.predict(X_train))
print("Train accuracy:", train_accuracy)
test_accuracy = accuracy_score(y_test, model.predict(X_test))
print("Test accuracy:" , test_accuracy)

ROC-AUC: 0.7875514993481095
MSE: 0.29193548387096774
Train accuracy: 0.7080645161290322
Test accuracy: 0.717948717948718


In [47]:
model = MLPClassifier(random_state=1, max_iter=300, hidden_layer_sizes= (10,5), activation = 'logistic', solver = 'adam')
model.fit(X_train, y_train)

predictions_train = model.predict(X_train)
roc_auc = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])
print("ROC-AUC:",roc_auc)

print("MSE:",mean_squared_error(y_train, predictions_train))

train_accuracy = accuracy_score(y_train, model.predict(X_train))
print("Train accuracy:", train_accuracy)
test_accuracy = accuracy_score(y_test, model.predict(X_test))
print("Test accuracy:" , test_accuracy)


ROC-AUC: 0.9158435462842243
MSE: 0.14516129032258066
Train accuracy: 0.8548387096774194
Test accuracy: 0.7692307692307693


/Users/mattcadel/Documents/Python/DSML/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


In [48]:
model = MLPClassifier(random_state=1, max_iter=300, hidden_layer_sizes= (10,5), activation = 'tanh', solver = 'adam')
model.fit(X_train, y_train)

predictions_train = model.predict(X_train)
roc_auc = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])
print("ROC-AUC:",roc_auc)

print("MSE:",mean_squared_error(y_train, predictions_train))

train_accuracy = accuracy_score(y_train, model.predict(X_train))
print("Train accuracy:", train_accuracy)
test_accuracy = accuracy_score(y_test, model.predict(X_test))
print("Test accuracy:" , test_accuracy)

ROC-AUC: 0.9148839634941328
MSE: 0.14516129032258066
Train accuracy: 0.8548387096774194
Test accuracy: 0.717948717948718


##### 3 - Further Experimentation

In [49]:
model = MLPClassifier(random_state=1, max_iter=1000, hidden_layer_sizes= (10,5), activation = 'logistic', solver = 'adam')
model.fit(X_train, y_train)

predictions_train = model.predict(X_train)
roc_auc = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])
print("ROC-AUC:",roc_auc)

print("MSE:",mean_squared_error(y_train, predictions_train))

train_accuracy = accuracy_score(y_train, model.predict(X_train))
print("Train accuracy:", train_accuracy)
test_accuracy = accuracy_score(y_test, model.predict(X_test))
print("Test accuracy:" , test_accuracy)

ROC-AUC: 0.9196818774445893
MSE: 0.1403225806451613
Train accuracy: 0.8596774193548387
Test accuracy: 0.7564102564102564


In [50]:
import xgboost as xgb

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

#  eta - set between 0 and 1 to help with weighting

param = {'max_depth': 4, 'eta': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = ['auc', 'rmse']
evallist = [(dtrain, 'train'), (dtest, 'eval')]

num_round = 20
bst = xgb.train(param, dtrain, num_round, evallist)

[0]	train-auc:0.91445	train-rmse:0.34414	eval-auc:0.82831	eval-rmse:0.40935
[1]	train-auc:0.93250	train-rmse:0.31752	eval-auc:0.83004	eval-rmse:0.41736
[2]	train-auc:0.94872	train-rmse:0.29910	eval-auc:0.84864	eval-rmse:0.40201
[3]	train-auc:0.96441	train-rmse:0.27501	eval-auc:0.86535	eval-rmse:0.39723
[4]	train-auc:0.97598	train-rmse:0.25496	eval-auc:0.86296	eval-rmse:0.39552
[5]	train-auc:0.98118	train-rmse:0.24066	eval-auc:0.85745	eval-rmse:0.40110
[6]	train-auc:0.98622	train-rmse:0.22857	eval-auc:0.85794	eval-rmse:0.40039
[7]	train-auc:0.98956	train-rmse:0.21531	eval-auc:0.86535	eval-rmse:0.39409
[8]	train-auc:0.99171	train-rmse:0.20518	eval-auc:0.86988	eval-rmse:0.39522
[9]	train-auc:0.99399	train-rmse:0.19185	eval-auc:0.86741	eval-rmse:0.39756
[10]	train-auc:0.99596	train-rmse:0.18144	eval-auc:0.86650	eval-rmse:0.39894
[11]	train-auc:0.99673	train-rmse:0.17219	eval-auc:0.86321	eval-rmse:0.40342
[12]	train-auc:0.99745	train-rmse:0.16503	eval-auc:0.86700	eval-rmse:0.40093
[13]	trai

/Users/mattcadel/Documents/Python/DSML/venv/lib/python3.10/site-packages/xgboost/core.py:617: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)
